---
# News Article Title Generation with LLMs

This Jupyter Notebook documents a **Natural Language Processing (NLP)** project focused on **Title Generation**, a sequence-to-sequence task where a news article's content is used to predict its title.

---
**Author Information:**
* **Author:** Radin Khayyam
* **Contact:** radinkhayyam@gmail.com

---
The project aims to systematically compare two powerful **Large Language Models (LLMs)**:
* **Gemma 3 - 4B - IT** (Instruction-Tuned)
* **Llama 3.1 - 8B - IT** (Instruction-Tuned)

---
## Project Steps

The notebook is structured into four main, distinct phases:

1.  **Data Loading & Exploration:** Load a public news article dataset and perform initial analysis of its structure and content statistics (e.g., length distribution of articles and titles).

2.  **Prompt-Based Evaluation (Zero-shot & Few-shot):** Evaluate the LLMs' inherent title generation capabilities by testing different **prompt engineering** strategies (zero-shot and few-shot) and analyzing their immediate impact on output quality.

3.  **Fine-Tuning with LoRA:** Enhance the LLMs' performance on this specific task by applying **Low-Rank Adaptation (LoRA)**, a parameter-efficient fine-tuning technique.

4.  **Inference and Evaluation:** Generate titles for the held-out test set using both the instruction-tuned (IT) and LoRA-fine-tuned models, and rigorously evaluate the results using industry-standard numerical metrics (e.g., ROUGE, BLEU).

---
## Goal

The ultimate goal is to determine the most effective strategy—direct instruction-tuning vs. LoRA fine-tuning—for optimizing these LLMs for the news title generation task and to establish a performance benchmark.

---

## Setup: Dependency Installation and Library Imports

This critical section establishes the project's computational environment. We install all necessary libraries for **Large Language Model (LLM)** handling, specifically focusing on the `unsloth` library for **parameter-efficient fine-tuning (PEFT)**. We install specific versions of `transformers` and `trl` for stability, along with a suite of **NLP evaluation metrics** (`rouge_score`, `bert_score`, `sacrebleu`, `nltk`) and text processing utilities (`hazm`). This ensures a robust and reproducible environment for the entire notebook.

In [2]:
%%capture
import os, re

# Install unsloth and required dependencies based on environment (Colab vs. local)
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Optimized installation for Google Colab with specific dependency versions for faster training
    import torch; v = re.match(r"[0-9\.]{3,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.32.post2" if v == "2.8.0" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth

# Install specific versions for general NLP and LLM operations
!pip install transformers==4.56.2
!pip install --no-deps trl==0.22.2

# Install libraries for popular NLP evaluation metrics
!pip install rouge_score
!pip install bert_score
!pip install nltk
!pip install sacrebleu

# Install text processing libraries, using --no-deps for specific control
!pip install hazm --no-deps
!pip install python-crfsuite flashtext

# Remove unnecessary embedding imports from hazm to prevent potential conflicts or excessive memory use
!sed -i '/from hazm.embedding import SentEmbedding/d' /usr/local/lib/python3.12/dist-packages/hazm/__init__.py
!sed -i '/from hazm.embedding import WordEmbedding/d' /usr/local/lib/python3.12/dist-packages/hazm/__init__.py

In [ ]:
# Core Data, ML, and Utility Libraries
import pandas as pd
import torch
import numpy as np
import gc # For garbage collection and memory management
from tqdm.auto import tqdm # For smart progress bars
import os
# osgeo_utils is typically for geospatial; removed for clean NLP context
from sklearn.model_selection import train_test_split # For data splitting

# Hugging Face and LLM-Specific Libraries
from transformers import (
    pipeline,
    AutoTokenizer,
    AutoModelForCausalLM,
)
from peft import (
    PeftModel,
    LoraConfig,
    get_peft_model
)
from datasets import Dataset # Hugging Face Dataset object for efficient data handling

# Unsloth and TRL for Efficient Fine-Tuning
from unsloth import FastLanguageModel
from trl import SFTConfig, SFTTrainer # Supervised Fine-Tuning Trainer
from unsloth.chat_templates import train_on_responses_only # Utility for chat template formatting

# Data Visualization and Rich Output
import matplotlib.pyplot as plt
import seaborn as sns
import textwrap # For formatting text output
from rich.console import Console
from rich.markdown import Markdown # For structured console output

# NLP Preprocessing and Evaluation Metrics
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction # BLEU metric calculation
from nltk.translate.meteor_score import meteor_score # METEOR metric calculation
from rouge_score import rouge_scorer # ROUGE metric calculation
from bert_score import score as bert_score_calc # BERTScore calculation
from hazm import Normalizer, word_tokenize # Farsi/Persian text normalization and tokenization
import sacrebleu # Robust standard for BLEU/chrf metrics

# Download NLTK data required for metrics (run silently)
nltk.download('punkt', quiet=True)
nltk.download('wordnet', quiet=True)
nltk.download('omw-1.4', quiet=True)

## Hugging Face Authentication

To enable downloading the proprietary **Gemma 3 - 4B - IT** and **Llama 3.1 - 8B - IT** models, which are gated on the Hugging Face Hub, **authentication is required**. This step uses the `huggingface_hub` library to prompt the user to securely log in using their valid personal access token. This is a prerequisite for subsequent model loading steps.

In [4]:
# If you want to load the models from Hugging Face Hub
# Authentication is required to access gated or proprietary models (e.g., Llama, Gemma)
from huggingface_hub import login
login()

## Data Acquisition and Stratified Split

Following authentication, the raw dataset is loaded from a local file path. The data is then immediately divided into training and testing sets. A **90/10 split** is applied using `train_test_split`, reserving 10% of the data for final inference and unbiased evaluation. Using a fixed `random_state` ensures the split is reproducible across multiple executions. This partitioning is essential to prevent data leakage and provide a robust measure of model generalization.

In [ ]:
# Load the dataset from the specified file path (assuming the file is available in the environment)
data = pd.read_excel("/content/Dataset.xlsx")

# Split the dataset into training and testing sets (90% for training, 10% for testing)
# Using a fixed random_state (42) ensures reproducibility of the split
train_data, test_data = train_test_split(data, test_size=0.1, random_state=42)

# Print the dimensions of the resulting datasets for verification
print(f"Training set shape: {train_data.shape}")
print(f"Test set shape: {test_data.shape}")

## Step 1: Exploratory Data Analysis (EDA)

This section performs **Exploratory Data Analysis (EDA)** to characterize the statistical properties of the dataset. Specifically, we analyze the **word count distributions** for both the article content and the corresponding titles. Understanding these distributions (average, standard deviation, and visual histograms) is crucial for:

1.  **Tokenization Strategy:** Informing decisions regarding maximum input and output sequence lengths for the LLMs.
2.  **Quality Assurance:** Identifying outliers or potential data entry errors (e.g., extremely short articles or excessively long titles).

We also calculate the **correlation** between content and title length to assess if longer articles tend to have longer titles, followed by a final check for **missing data** and a presentation of **random samples** to ground the numerical analysis in the actual text.

In [ ]:
# Calculate the number of words in the 'content' and 'title' columns using string splitting
data['content_length'] = data['content'].str.split().str.len()
data['title_length'] = data['title'].str.split().str.len()

# Calculate the mean and standard deviation for the newly created length columns
avg_content_length = data['content_length'].mean()
std_content_length = data['content_length'].std()
avg_title_length = data['title_length'].mean()
std_title_length = data['title_length'].std()

# Print key summary statistics
print(f"Average Content Word Count: {avg_content_length:.2f}")
print(f"Standard Deviation of Content Word Count: {std_content_length:.2f}")
print(f"Average Title Word Count: {avg_title_length:.2f}")
print(f"Standard Deviation of Title Word Count: {std_title_length:.2f}")

# Display comprehensive descriptive statistics for the length columns
display(data[['content_length', 'title_length']].describe())

# Visualize the distribution of content length using a histogram with KDE
plt.figure(figsize=(10, 6))
sns.histplot(data['content_length'], kde=True, bins=50)
plt.title('Distribution of Content Word Count')
plt.xlabel('Word Count')
plt.ylabel('Frequency')
plt.show()

# Visualize the distribution of title length using a histogram with KDE
plt.figure(figsize=(10, 6))
sns.histplot(data['title_length'], kde=True, bins=50)
plt.title('Distribution of Title Word Count')
plt.xlabel('Word Count')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# Calculate the Pearson correlation coefficient between article content length and title length
correlation_coefficient = data['content_length'].corr(data['title_length'])
print(f"Correlation between Content Length and Title Length: {correlation_coefficient:.2f}")

# Visualize the relationship between content and title length using a regression plot
plt.figure(figsize=(10, 6))
# Use alpha=0.3 to better visualize density of overlapping points
sns.regplot(x='content_length', y='title_length', data=data, scatter_kws={'alpha':0.3})
plt.title('Scatter Plot of Content Length vs. Title Length')
plt.xlabel('Content Word Count')
plt.ylabel('Title Word Count')
plt.grid(True)
plt.show()

In [ ]:
# Final check for any remaining missing values across all columns
print("\nMissing values per column:")
display(data.isnull().sum())

In [ ]:
# Select 5 random samples from the training dataset for manual inspection
random_samples = train_data.sample(5, random_state=42) # Use fixed state for reproducibility

# Iterate through the random samples and print the title and content, wrapped for readability
for index, row in random_samples.iterrows():
  # Wrap the text to a standard width (60 characters) for cleaner display
  wrapped_content = textwrap.fill(row['content'], width=90) # Increased width for better article content display
  wrapped_title = textwrap.fill(row['title'], width=90)
  print('Title:')
  print(wrapped_title + '\n')
  print('Content:')
  print(wrapped_content)
  print("-" * 50)

## Step 2: Prompt-Based Evaluation - Defining Instruction Templates

This section defines four distinct instruction templates used to benchmark the zero-shot and few-shot capabilities of the instruction-tuned LLMs. These prompts progressively increase in complexity and constraints to assess their impact on output quality in the context of Persian news title generation:

1.  **`simple_prompt`**: A minimal instruction for a direct, unguided baseline evaluation.
2.  **`zero_shot_prompt`**: A detailed, **persona-based instruction** that imposes explicit constraints on the output (language, length, format, focus) without examples.
3.  **`few_shot_prompt`**: Expands on the zero-shot template by incorporating five high-quality example pairs, guiding the model toward the expected output style.
4.  **`refined_prompt`**: This is the most specialized template. It includes **explicit domain-specific guidance** that, for fact-check articles, the generated title **must summarize the original claim (شرح ادعا)**, regardless of the subsequent verification (نتیجه بررسی). This is crucial, as the examples demonstrate that the title should report the claim itself, even if it is incorrect, and must **not** include the verification outcome.

## **Prompts**

In [7]:
# --- 1. Simple, Minimalist Prompt Template (Zero-Shot Baseline) ---
# A concise, direct instruction for title generation with minimal constraints.
simple_prompt = """
Generate a high-quality headline for the following news report, responding only with the Persian title and no other text.

News report:
"""

# --- 2. Detailed, Persona-Based Prompt Template (Zero-Shot) ---
# Establishes an expert persona and formal constraints to maximize zero-shot output quality.
zero_shot_prompt = """You are an expert-level, highly experienced Persian news editor. You have a deep understanding of journalistic standards, factual reporting, and the nuances of the Persian language.

Your task is to generate a single, high-quality, informative headline (title) for a given news report. You will be provided with the full text of the news content.

## **Instructions and Constraints:**

* **Language**: The generated title **MUST** be written primarily in the Persian language. English words should only be used if they are specific proper nouns (such as organization names, brands, or technical terms) **and** they appear in the original news content provided.
* **Length**: The title must be approximately **15 words** long.
* **Format**: The title **MUST** be a **declarative sentence**. It should state a fact or summarize the main point directly. It must **not** be phrased as a question.
* **Focus**: The primary goal is to be **informative and accurate**. The title must be 100% relevant to the provided news content and precisely summarize its core subject or most significant finding.

Analyze the provided news content, identify its most critical information, and then craft a single headline that meets all of the above criteria.

## **Output Format:**

Respond **only** with the generated Persian title. Do not add any commentary, explanations, or introductory text.

News report:
"""

# --- 3. Few-Shot Prompt Template (Examples without explicit domain logic) ---
# Extends the Zero-Shot prompt with five examples to guide the model's output style.
few_shot_prompt = """You are an expert-level, highly experienced Persian news editor. You have a deep understanding of journalistic standards, factual reporting, and the nuances of the Persian language.

Your task is to generate a single, high-quality, informative headline (title) for a given news report. You will be provided with the full text of the news content.

## **Instructions and Constraints:**

* **Language**: The generated title **MUST** be written primarily in the Persian language. English words should only be used if they are specific proper nouns (such as organization names, brands, or technical terms) **and** they appear in the original news content provided.
* **Length**: The title must be approximately **15 words** long.
* **Format**: The title **MUST** be a **declarative sentence**. It should state a fact or summarize the main point directly. It must **not** be phrased as a question.
* **Focus**: The primary goal is to be **informative and accurate**. The title must be 100% relevant to the provided news content and precisely summarize its core subject or most significant finding.

Analyze the provided news content, identify its most critical information, and then craft a single headline that meets all of the above criteria.

## **Examples:**

Here are five examples of correct `Content` to `Title` generation:

---
**Example 1**

**Content:**
شرح ادعا کاربری در ایکس ادعا کرده است که بیش از هزار کارگر
طی شش ماه در حوادث ضمن کار جان خود را از دست داده‌اند. نتیجه
بررسی براساس آمار اختصاصی سازمان پزشکی قانونی : در شش ماهه
نخست سال جاری ۱۰۷۷ نفر در حوادث کار جان خود را از دست دادند.
این رقم در مقایسه با مدت مشابه سال قبل با آمار فوتی ۹۳۱ نفر،
۱۵/۷ درصد افزایش داشته است. آمار اختصاصی سازمان پزشکی قانونی
در سایت باشگاه خبرنگاران جوان براساس سالنامه آماری وزارت کار
در سال ۱۴۰۱ : تعداد متوفیان حوادث ناشی از کار ثبت شده در
مراکز پزشکی قانونی در سال ۱۴۰۱ ، ۱۹۰۰ نفر بوده است. سالنامه
آماری وزارت کار در سال ۱۴۰۱

**Title:**
بیش از هزار کارگر طی شش ماه در حوادث ضمن کار کشته شده‌اند

---
**Example 2**

**Content:**
حسن روحانی در آیین افتتاح طرح‌های ملی وزارت جهاد کشاورزی، در
هشتاد و دومین پویش تدبیر و امید برای جهش تولید گفت: « مردم
خوزستان علی‌رغم اینکه توصیه ما این بود که آنجا برنج‌کاری
نشود، اما سال‌هاست برنج‌کاری هم انجام می‌دهند که خودِ آن هم
مزید بر علت شده و این‌ها همه دست به دست هم داده و مردم عزیز
در سختی قرار گرفتند.» صفحه شبکه منوتو در اینستاگرام به نقل
از حسن روحانی نوشته است: «مردم خوزستان علی‌Rateرغم اینکه توصیه
ما این بود که در آنجا برنج‌کاری نشود، اما سال‌هاست برنج‌کاری
هم انجام می‌ده دهند که خود آن هم مزید بر علت در بحران کم‌آبی
شده است.» به گزارش سایت رسمی پایگاه اطلاع رسانی ریاست جمهوری
به تاریخ ۲۸ تیرماه، حسن روحانی در آیین افتتاح طرح‌های ملی
وزارت جهاد کشاورزی در هشتاد و دومین پویش تدبیر و امید برای
جهش تولید گفت: «مردم خوزستان علی‌رغم اینکه توصیه ما این بود
که آنجا برنج‌کاری نشود؛ اما سال‌هاست برنج‌کاری هم انجام
می‌دهند که خودِ آن هم مزید بر علت شده و این‌ها همه دست به
دست هم داده و مردم عزیز در سختی قرار گرفتند.» سایت رسمی
پایگاه اطلاع رسانی ریاست جمهوری

**Title:**
حسن روحانی گفته‌است: «برنج‌کاری مردم خوزستان از دلایل بحران
کم‌آبی است.»

---
**Example 3**

**Content:**
شرح ادعا کاربری در شبکه اجتماعی ایکس ادعا کرده‌است که فقط در
۸ ماه، ۴ هزار و ۵۰۰ پزشک و پرستار از ایران مهاجرت کرده‌اند.
نتیجه بررسی این آمار، براساس گزارش روزنامه ایران از شمار
گواهی‌های صادر شده گود استندینگ (Good standing) صادر شده
توسط سازمان نظام پزشکی است. گواهی گود استندینگ یکی از مدارکی
است که اعضای کادر درمان باید برای مهاجTرت و فعالیت حرفه‌ای در
کشورهای خارجی ارائه دهند؛ این گواهی حسن سابقه افراد را مشخص
می‌کند و یعنی این افراد صلاحیت لازم را برای فعالیت در کشور
مقصد را دارند. بر اساس این گزارش، در 8 ماه ابتدایی امسال،
4500 نفر گواهی گود استندینگ دریافت کرده‌اند، اما صرف دریافت
این گواهی به معنای مهاجTرت قطعی نیست. بنابراین، اگرچه دریافت
گواهی گوداستندینگ برای اقدام در راستای مهاجرت است؛ اما به
این معنا که هر کس این گواهی را دریافت کرده، سپس مهاجرت
نموده، نیست. پس ادعای این کاربر گمراه‌کننده ارزیابی می‌شود.

**Title:**
در 8 ماه ابتدایی سال جاری، 4500 پزشک و پرستار مهاجرت
کرده‌اند

---
**Example 4**

**Content:**
گرفتن حق اشتراک و پخش آگهی قبل از انتشار محتوا توسط
پلتفرم‌های اینترنتی در هیچ جای دنیا مرسوم نیست و به زودی
برای آن دستورالعمل قانونی نوشته و پس از مصوب‌ شدن، به آن‌ها
ابلاغ می‌شود. در حال حاضر، قانونی که پلتفرم‌ها را از این عمل
منع کند وجود ندارد. مصطفی قاسمیان، روزنامه‌نگار سینما و
تلویزیون، در توییتر ادعا کرده است: «با وجودی که ۵ ماه پیش
ساترا صراحتا اعلام کرد پلتفرم‌هایی که هزینه اشتراک می‌گیرند،
حق نمایش آگهی پیش از پخش محتوا ندارند، اما فیلیمو و نماوا
همچنان این عمل غیرقانونی را انجام می‌دهند.» توییت قاسمیان
درباره پخش آگهی فیلیمو و نماوا علی سعد، معاون کاربران و
تنظیم مقررات اجتماعی ساترا (سازمان تنظیم مقررات صوت و تصویر
فراگیر در فضای مجازی) در تماس تلفنی با فکت‌یار گفت : «ما
فروردین ماه اعلام کردیم... در حال حاضر
این دو پلتفرم کار غیر‌قانونی انجام نمی‌دهند چون هنوز قانون
آن مصوب و ابلاغ نشده است.»

**Title:**
با وجود اعلام ساترا مبنی بر اینکه پلتفرم‌هایی که هزینه
اشتراک می‌گیرند، حق نمایش آگهی پیش از پخش محتوا را ندارند،
فیلیمو و نماوا با انتشار آگهی قبل از نمایش محتوا عمل
غیرقانونی انجام می‌دهند.

---
**Example 5**

**Content:**
شرح ادعا عبدالرضا داوری با انتشار فیلمی از مردی که برای
دریافت هزینه اشتراک تلویزیون BBC به خانه یکی از شهروندان این
کشور رفته‌است، گفت: «در حالیکه مردم بریتانیا مجبور هستند
برای شبکه BBC حق اشتراک بپردازند، شبکه BBC چه منافعی در
ایران دارد که برنامه‌های بخش فارسی آن، بصورت رایگان و ۲۴
ساعته برای ملت ایران پخش می‌شود؟». نتیجه بررسی براساس
آیین‌نامه‌ای که در وبسایت دولت بریتانیا برای دریافت مجوز
دیدن تلویزیون وجود دارد ؛ شهروندان برای دیدن برنامه‌های زنده
تلویزیون، به صورت کلی و برای همه شبکه‌ها (اعم از BBC)، باید
۱۶۹.۵پوند پرداخت کنند. دیدن بی بی سی فارسی، به صورت آنلاین
یا با استفاده از آنتن‌های ماهواره، برای مخاطبان رایگان است.
بنابراین، ادعای عبدالرضا داوری در مورد اینکه مردم بریتانیا
باید برای دیدن BBC اشتراک تهیه کنند اما این شبکه به طور
رایگان برای مردم ایران به طور 24 ساعته برنامه‌سازی می‌کند،
درست ارزیابی می‌شود.

**Title:**
مردم بریتانیا باید برای دیدن BBC اشتراک بخرند؛ در حالی که
دیدن BBC فارسی برای مردم ایران رایگان است

---

## **Output Format:**

Respond **only** with the generated Persian title. Do not add any commentary, explanations, or introductory text.

News report:
"""

# --- 4. Highly Refined Prompt Template (With Explicit Domain Logic) ---
refined_prompt = """
You are an expert-level, highly experienced Persian news editor. Your task is to generate a single, high-quality, informative headline (title) based on a given news report.

The content you will receive is a fact-check, which contains two main parts:
1.  A **"شرح ادعا" (Claim Description)**
2.  A **"نتیجه بررسی" (Verification / Result)**

## **Core Logic: Handling Claims vs. Verifications**

Your primary task is to identify the most newsworthy part of the article to summarize as a headline.

* The content you receive is a fact-check. In this format, the **"Claim" (ادعا)** is almost always the main subject and the most newsworthy part.
* **Crucial Instruction:** In many cases, even when the **"Verification" (نتیجه بررسی)** finds that the "Claim" is **incorrect (نادرست)** or **misleading (گمراه‌کننده)**, the claim is *still* the main subject of the article.
* **In these specific cases, your headline MUST be a declarative statement of that incorrect or semi-correct claim.**
* The examples (especially Example 3 and Example 4) clearly demonstrate this rule. In both, the verification finds the claim to be false or misleading, but the correct `Title` is still a statement of the `Claim`. **Do not include the verification outcome (e.g., 'incorrect', 'false') in the final title.**

## **Instructions and Constraints:**

* **Language**: The generated title **MUST** be written primarily in the Persian language. English words should only be used if they are specific proper nouns (such as organization names, brands, or technical terms) **and** they appear in the original "شرح ادعا".
* **Length**: The title must be approximately **15 words** long.
* **Format**: The title **MUST** be a **declarative sentence**. It must **not** be phrased as a question.

Analyze the provided news content, identify the core claim, and then craft a single headline that meets all of the above criteria based on the Core Logic.

## **Examples:**

Here are five examples showing how the `Content` (which includes a claim and a verification) is used to create a `Title` (which is *only* the claim).

---
**Example 1**

**Content:**
شرح ادعا کاربری در ایکس ادعا کرده است که بیش از هزار کارگر
طی شش ماه در حوادث ضمن کار جان خود را از دست داده‌اند. نتیجه
بررسی براساس آمار اختصاصی سازمان پزشکی قانونی : در شش ماهه
نخست سال جاری ۱۰۷۷ نفر در حوادث کار جان خود را از دست دادند.
این رقم در مقایسه با مدت مشابه سال قبل با آمار فوتی ۹۳۱ نفر،
۱۵/۷ درصد افزایش داشته است. آمار اختصاصی سازمان پزشکی قانونی
در سایت باشگاه خبرنگاران جوان براساس سالنامه آماری وزارت کار
در سال ۱۴۰۱ : تعداد متوفیان حوادث ناشی از کار ثبت شده در
مراکز پزشکی قانونی در سال ۱۴۰۱ ، ۱۹۰۰ نفر بوده است. سالنامه
آماری وزارت کار در سال ۱۴۰۱

**Title:**
بیش از هزار کارگر طی شش ماه در حوادث ضمن کار کشته شده‌اند

---
**Example 2**

**Content:**
حسن روحانی در آیین افتتاح طرح‌های ملی وزارت جهاد کشاورزی، در
هشتاد و دومین پویش تدبیر و امید برای جهش تولید گفت: « مردم
خوزستان علی‌رغم اینکه توصیه ما این بود که آنجا برنج‌کاری
نشود، اما سال‌هاست برنج‌کاری هم انجام می‌دهند که خودِ آن هم
مزید بر علت شده و این‌ها همه دست به دست هم داده و مردم عزیز
در سختی قرار گرفتند.» صفحه شبکه منوتو در اینستاگرام به نقل
از حسن روحانی نوشته است: «مردم خوزستان علی‌Rateرغم اینکه توصیه
ما این بود که در آنجا برنج‌کاری نشود، اما سال‌هاست برنج‌کاری
هم انجام می‌ده دهند که خود آن هم مزید بر علت در بحران کم‌آبی
شده است.» به گزارش سایت رسمی پایگاه اطلاع رسانی ریاست جمهوری
به تاریخ ۲۸ تیرماه، حسن روحانی در آیین افتتاح طرح‌های ملی
وزارت جهاد کشاورزی در هشتاد و دومین پویش تدبیر و امید برای
جهش تولید گفت: «مردم خوزستان علی‌رغم اینکه توصیه ما این بود
که آنجا برنج‌کاری نشود؛ اما سال‌هاست برنج‌کاری هم انجام
می‌دهند که خودِ آن هم مزید بر علت شده و این‌ها همه دست به
دست هم داده و مردم عزیز در سختی قرار گرفتند.» سایت رسمی
پایگاه اطلاع رسانی ریاست جمهوری

**Title:**
حسن روحانی گفته‌است: «برنج‌کاری مردم خوزستان از دلایل بحران
کم‌آبی است.»

---
**Example 3**

**Content:**
شرح ادعا کاربری در شبکه اجتماعی ایکس ادعا کرده‌است که فقط در
۸ ماه، ۴ هزار و ۵۰۰ پزشک و پرستار از ایران مهاجرت کرده‌اند.
نتیجه بررسی این آمار، براساس گزارش روزنامه ایران از شمار
گواهی‌های صادر شده گود استندینگ (Good standing) صادر شده
توسط سازمان نظام پزشکی است. گواهی گود استندینگ یکی از مدارکی
است که اعضای کادر درمان باید برای مهاجTرت و فعالیت حرفه‌ای در
کشورهای خارجی ارائه دهند؛ این گواهی حسن سابقه افراد را مشخص
می‌کند و یعنی این افراد صلاحیت لازم را برای فعالیت در کشور
مقصد را دارند. بر اساس این گزارش، در 8 ماه ابتدایی امسال،
4500 نفر گواهی گود استندینگ دریافت کرده‌اند، اما صرف دریافت
این گواهی به معنای مهاجTرت قطعی نیست. بنابراین، اگرچه دریافت
گواهی گوداستندینگ برای اقدام در راستای مهاجرت است؛ اما به
این معنا که هر کس این گواهی را دریافت کرده، سپس مهاجرت
نموده، نیست. پس ادعای این کاربر گمراه‌کننده ارزیابی می‌شود.

**Title:**
در 8 ماه ابتدایی سال جاری، 4500 پزشک و پرستار مهاجرت
کرده‌اند

---
**Example 4**

**Content:**
گرفتن حق اشتراک و پخش آگهی قبل از انتشار محتوا توسط
پلتفرم‌های اینترنتی در هیچ جای دنیا مرسوم نیست و به زودی
برای آن دستورالعمل قانونی نوشته و پس از مصوب‌ شدن، به آن‌ها
ابلاغ می‌شود. در حال حاضر، قانونی که پلتفرم‌ها را از این عمل
منع کند وجود ندارد. مصطفی قاسمیان، روزنامه‌نگار سینما و
تلویزیون، در توییتر ادعا کرده است: «با وجودی که ۵ ماه پیش
ساترا صراحتا اعلام کرد پلتفرم‌هایی که هزینه اشتراک می‌گیرند،
حق نمایش آگهی پیش از پخش محتوا ندارند، اما فیلیمو و نماوا
همچنان این عمل غیرقانونی را انجام می‌دهند.» توییت قاسمیان
درباره پخش آگهی فیلیمو و نماوا علی سعد، معاون کاربران و
تنظیم مقررات اجتماعی ساترا (سازمان تنظیم مقررات صوت و تصویر
فراگیر در فضای مجازی) در تماس تلفنی با فکت‌یار گفت : «ما
فروردین ماه اعلام کردیم... در حال حاضر
این دو پلتفرم کار غیر‌قانونی انجام نمی‌دهند چون هنوز قانون
آن مصوب و ابلاغ نشده است.»

**Title:**
با وجود اعلام ساترا مبنی بر اینکه پلتفرم‌هایی که هزینه
اشتراک می‌گیرند، حق نمایش آگهی پیش از پخش محتوا را ندارند،
فیلیمو و نماوا با انتشار آگهی قبل از نمایش محتوا عمل
غیرقانونی انجام می‌دهند.

---
**Example 5**

**Content:**
شرح ادعا عبدالرضا داوری با انتشار فیلمی از مردی که برای
دریافت هزینه اشتراک تلویزیون BBC به خانه یکی از شهروندان این
کشور رفته‌است، گفت: «در حالیکه مردم بریتانیا مجبور هستند
برای شبکه BBC حق اشتراک بپردازند، شبکه BBC چه منافعی در
ایران دارد که برنامه‌های بخش فارسی آن، بصورت رایگان و ۲۴
ساعته برای ملت ایران پخش می‌شود؟». نتیجه بررسی براساس
آیین‌نامه‌ای که در وبسایت دولت بریتانیا برای دریافت مجوز
دیدن تلویزیون وجود دارد ؛ شهروندان برای دیدن برنامه‌های زنده
تلویزیون، به صورت کلی و برای همه شبکه‌ها (اعم از BBC)، باید
۱۶۹.۵پوند پرداخت کنند. دیدن بی بی سی فارسی، به صورت آنلاین
یا با استفاده از آنتن‌های ماهواره، برای مخاطبان رایگان است.
بنابراین، ادعای عبدالرضا داوری در مورد اینکه مردم بریتانیا
باید برای دیدن BBC اشتراک تهیه کنند اما این شبکه به طور
رایگان برای مردم ایران به طور 24 ساعته برنامه‌سازی می‌کند،
درست ارزیابی می‌شود.

**Title:**
مردم بریتانیا باید برای دیدن BBC اشتراک بخرند؛ در حالی که
دیدن BBC فارسی برای مردم ایران رایگان است

---

## **Output Format:**

Respond **only** with the generated Persian title. Do not add any commentary, explanations, or introductory text.

News report:
"""

## Step 2.1: Qualitative Evaluation with Gemma 3 – 4B – IT

This phase loads the **Gemma 3 – 4B – IT** model and its tokenizer for initial zero-shot and few-shot inference. Since instruction-tuned models often require specific input schemas, a helper function is defined to format the article content and the instruction template according to Gemma's required **`chat_template`**.

We then execute a **qualitative analysis** by applying the four defined prompt strategies (`simple`, `zero-shot`, `few-shot`, `refined`) to a small, fixed subset of the test data (15 samples). The generated titles are compared against the ground truth titles for a preliminary assessment of each prompt's efficacy. The LLM parameters are set to allow for diverse generation (**sampling**) while limiting the output length to focus on the title.

In [ ]:
# Define the Hugging Face ID for the Gemma model
GEMMA_3_4B_IT = "google/gemma-3-4b-it"

# Load the Gemma 3-4B-IT model, using bfloat16 for efficiency and VRAM conservation
# Device mapping 'auto' handles parallel loading across available GPUs
gemma_3_4b_it_model = AutoModelForCausalLM.from_pretrained(
    GEMMA_3_4B_IT,
    dtype=torch.bfloat16,
    device_map="auto",
    low_cpu_mem_usage=True,
)

# Load the corresponding tokenizer and set the pad token for batched generation readiness
gemma_3_4b_it_tokenizer = AutoTokenizer.from_pretrained(GEMMA_3_4B_IT)
if gemma_3_4b_it_tokenizer.pad_token is None:
    gemma_3_4b_it_tokenizer.pad_token = gemma_3_4b_it_tokenizer.eos_token

# Create a text generation pipeline for streamlined inference
gemma_3_4b_it_pipe = pipeline(
    task="text-generation",
    model=gemma_3_4b_it_model,
    tokenizer=gemma_3_4b_it_tokenizer,
    dtype=torch.bfloat16
)

# Define generation parameters (kwargs) to be used consistently across all prompt tests
gemma_3_4b_it_pipe_kwargs = {
    "max_new_tokens": 128,  # Limit output length, as titles are relatively short
    "return_full_text": False,  # Return only the generated text, not the prompt
    "do_sample": True,  # Enable sampling for varied, less deterministic output
    "temperature": 0.7,  # Moderate temperature for balanced creativity and coherence
    "top_p": 0.9  # Set high top_p for broad token consideration
}

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/90.6k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.64G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
def format_for_gemma(system_prompt_text, query_text):
    """
    Formats the instructional system prompt and the article content into the
    Gemma model's required conversation structure (chat_template).

    Args:
        system_prompt_text (str): The specific instruction template (e.g., zero_shot_prompt).
        query_text (str): The news article content to be summarized.

    Returns:
        list: A list of dictionaries conforming to the Hugging Face `messages` format.
    """
    # Note: Gemma models often combine system and user instruction within the 'user' role,
    # but using this structure is robust for instruction tuning models.
    messages = [
        {
            "role": "system",
            "content": [{"type": "text", "text": system_prompt_text}]
        },
        {
            "role": "user",
            "content": [{"type": "text", "text": query_text}]
        }
    ]
    return messages

In [ ]:
# Select a small, reproducible subset of the test data for qualitative inspection
random_samples_for_evaluation = test_data.sample(15, random_state=77)

# Initialize console for formatted, colored output display
console = Console()

# Iterate through the selected samples and generate titles using all four prompt strategies
for index, row in random_samples_for_evaluation.iterrows():
    sample_content = row['content']
    correct_title = row['title']

    # --- Simple Prompt Generation ---
    formatted_simple_prompt = format_for_gemma(simple_prompt, sample_content)
    simple_generated_title = gemma_3_4b_it_pipe(formatted_simple_prompt, **gemma_3_4b_it_pipe_kwargs)[0]["generated_text"]

    # --- Zero-Shot Prompt Generation ---
    formatted_zero_shot_prompt = format_for_gemma(zero_shot_prompt, sample_content)
    zero_shot_generated_title = gemma_3_4b_it_pipe(formatted_zero_shot_prompt, **gemma_3_4b_it_pipe_kwargs)[0]["generated_text"]

    # --- Few-Shot Prompt Generation ---
    formatted_few_shot_prompt = format_for_gemma(few_shot_prompt, sample_content)
    few_shot_generated_title = gemma_3_4b_it_pipe(formatted_few_shot_prompt, **gemma_3_4b_it_pipe_kwargs)[0]["generated_text"]

    # --- Refined Prompt Generation (Includes domain-specific logic) ---
    formatted_refined_prompt = format_for_gemma(refined_prompt, sample_content)
    refined_generated_title = gemma_3_4b_it_pipe(formatted_refined_prompt, **gemma_3_4b_it_pipe_kwargs)[0]["generated_text"]

    # Display results for side-by-side comparison
    console.print(f"[yellow]--- Sample {index} ---[/yellow]")
    console.print(f"[green]Original Content:[/green] {textwrap.fill(sample_content, width=80)}\n")
    console.print(f"[green]Actual Title (Dataset):[/green] {correct_title}\n")
    console.print(f"[deep_sky_blue1]Generated Title (Gemma - Simple Prompt):[/deep_sky_blue1] {simple_generated_title}")
    console.print(f"[deep_sky_blue1]Generated Title (Gemma - Zero-Shot Prompt):[/deep_sky_blue1] {zero_shot_generated_title}")
    console.print(f"[deep_sky_blue1]Generated Title (Gemma - Few-Shot Prompt):[/deep_sky_blue1] {few_shot_generated_title}")
    console.print(f"[deep_sky_blue1]Generated Title (Gemma - Refined Prompt):[/deep_sky_blue1] {refined_generated_title}\n")
    console.print("-" * 100)

## 🤖 Step 2.2: Qualitative Evaluation with Llama 3.1 – 8B – IT

This phase mirrors the previous step, focusing on the **Llama 3.1 – 8B – IT** model. Given the larger parameter count (8B vs. 4B), **memory management** is critical, so we explicitly clear the CUDA memory of the previously loaded Gemma model.

We load the Llama model, define a specific function to format inputs according to the Llama instruction template, and then perform the same **qualitative analysis** on the fixed 15 test samples. The generated titles from Llama across the four prompt strategies are then displayed for visual comparison against Gemma's performance.

In [ ]:
# --- 1. Cleanup Memory from Previous Model (Gemma) ---
print("Gemma model inference complete. Explicitly clearing memory...")
# Delete model, pipeline, and tokenizer objects to free VRAM
del gemma_3_4b_it_model, gemma_3_4b_it_pipe, gemma_3_4b_it_tokenizer
# Run Python garbage collection and empty the CUDA cache
gc.collect()
torch.cuda.empty_cache()
print("Memory cleared successfully.")

In [ ]:
# --- 2. Load Llama 3.1 Model and Setup Pipeline ---
# Define the Hugging Face ID for the Llama model
Llama_31_8B_IT = "meta-llama/Meta-Llama-3.1-8B-Instruct"

# Load the Llama 3.1-8B-IT model, using bfloat16 and automatic device mapping
llama_31_8b_it_model = AutoModelForCausalLM.from_pretrained(
    Llama_31_8B_IT,
    dtype=torch.bfloat16,
    device_map="auto",
    low_cpu_mem_usage=True, # Reduces CPU overhead during loading
)

# Load the corresponding tokenizer and ensure padding token is defined
llama_31_8b_it_tokenizer = AutoTokenizer.from_pretrained(Llama_31_8B_IT)
if llama_31_8b_it_tokenizer.pad_token is None:
    llama_31_8b_it_tokenizer.pad_token = llama_31_8b_it_tokenizer.eos_token

# Create a text generation pipeline for streamlined inference
llama_31_8b_it_pipe = pipeline(
    task="text-generation",
    model=llama_31_8b_it_model,
    tokenizer=llama_31_8b_it_tokenizer,
    dtype=torch.bfloat16
)

# Define generation parameters (kwargs) for consistent inference.
# Slightly higher temperature (0.8) used here compared to Gemma (0.7) for potential comparison diversity.
llama_31_8b_it_pipe_kwargs = {
    "max_new_tokens": 128,  # Limit output length
    "return_full_text": False,  # Return only the generated text
    "do_sample": True,  # Enable sampling
    "temperature": 0.8,
    "top_p": 0.9
}

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
def format_for_llama(system_prompt_text, query_text):
    """
    Formats the instructional system prompt and the article content into the
    Llama model's required conversation structure (chat_template).

    Args:
        system_prompt_text (str): The specific instruction template.
        query_text (str): The news article content.

    Returns:
        list: A list of dictionaries conforming to the Hugging Face `messages` format,
              suitable for Llama instruction-tuned models.
    """
    messages = [
        {
            "role": "system",
            "content": system_prompt_text
        },
        {
            "role": "user",
            "content": query_text
        }
    ]
    return messages

In [ ]:
# --- 4. Run Qualitative Inference on Llama 3.1 ---
# Use the same 15 random samples selected previously (fixed random_state=77)
random_samples_for_evaluation = test_data.sample(15, random_state=77)

console = Console()

# Iterate through the selected samples and generate titles using all four prompt strategies
for index, row in random_samples_for_evaluation.iterrows():
    sample_content = row['content']
    correct_title = row['title']

    # --- Simple Prompt Generation (Llama) ---
    formatted_simple_prompt = format_for_llama(simple_prompt, sample_content)
    simple_generated_title = llama_31_8b_it_pipe(formatted_simple_prompt, **llama_31_8b_it_pipe_kwargs)[0]["generated_text"]

    # --- Zero-Shot Prompt Generation (Llama) ---
    formatted_zero_shot_prompt = format_for_llama(zero_shot_prompt, sample_content)
    zero_shot_generated_title = llama_31_8b_it_pipe(formatted_zero_shot_prompt, **llama_31_8b_it_pipe_kwargs)[0]["generated_text"]

    # --- Few-Shot Prompt Generation (Llama) ---
    formatted_few_shot_prompt = format_for_llama(few_shot_prompt, sample_content)
    few_shot_generated_title = llama_31_8b_it_pipe(formatted_few_shot_prompt, **llama_31_8b_it_pipe_kwargs)[0]["generated_text"]

    # --- Refined Prompt Generation (Llama) ---
    formatted_refined_prompt = format_for_llama(refined_prompt, sample_content)
    refined_generated_title = llama_31_8b_it_pipe(formatted_refined_prompt, **llama_31_8b_it_pipe_kwargs)[0]["generated_text"]

    # Display results for side-by-side comparison
    console.print(f"[yellow]--- Sample {index} ---[/yellow]")
    console.print(f"[green]Original Content:[/green] {textwrap.fill(sample_content, width=80)}\n")
    console.print(f"[green]Actual Title (Dataset):[/green] {correct_title}\n")
    console.print(f"[deep_sky_blue1]Generated Title (Llama - Simple Prompt):[/deep_sky_blue1] {simple_generated_title}")
    console.print(f"[deep_sky_blue1]Generated Title (Llama - Zero-Shot Prompt):[/deep_sky_blue1] {zero_shot_generated_title}")
    console.print(f"[deep_sky_blue1]Generated Title (Llama - Few-Shot Prompt):[/deep_sky_blue1] {few_shot_generated_title}")
    console.print(f"[deep_sky_blue1]Generated Title (Llama - Refined Prompt):[/deep_sky_blue1] {refined_generated_title}\n")
    console.print("-" * 100)

## Step 3: Fine-Tuning The Models using LoRA

This major phase involves specializing the pre-trained instruction-tuned LLMs (**Gemma 3 – 4B – IT** and **Llama 3.1 – 8B – IT**) for the target task: Persian news title generation. We use **Low-Rank Adaptation (LoRA)**, a Parameter-Efficient Fine-Tuning (PEFT) technique, to significantly reduce computational overhead by only training a small fraction of new, low-rank parameters (adapters) while keeping the massive weights of the base model frozen.

This process consists of the following critical sub-steps:
1.  **Configuration and Model Loading:** Define model paths and load the base model/tokenizer.
2.  **LoRA Adapter Attachment:** Attach a new LoRA configuration to the base model.
3.  **Data Preparation:** Convert the raw text pairs into the model's required multi-turn chat format (`system`, `user`, `assistant`).
4.  **SFTTrainer Configuration:** Use the `trl.SFTTrainer` for supervised instruction tuning with specialized training arguments.
5.  **Training and Saving:** Execute the training loop and save the resulting LoRA adapter weights.

## Step 3.1: Fine-Tuning Gemma 3 – 4B – IT using LoRA


In [8]:
# --- Configuration ---
# Define the Hugging Face ID for the base instruction-tuned model
GEMMA_3_4B_IT = "google/gemma-3-4b-it"

# Define the output directory for saving the LoRA adapter checkpoints
# Ensure this path is valid, typically pointing to Google Drive in Colab
ADAPTER_OUTPUT_DIR = "/content/drive/MyDrive/Colab Notebooks/Radin/Gemma_Output"

In [9]:
# --- Load The Base Model ---
# Load the pre-trained Gemma model using bfloat16 precision and auto device mapping
gemma_3_4b_it_model = AutoModelForCausalLM.from_pretrained(
    GEMMA_3_4B_IT,
    dtype=torch.bfloat16,
    device_map="auto",
    low_cpu_mem_usage=True, # Reduces CPU memory usage during loading
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# --- Load The Tokenizer ---
gemma_3_4b_it_tokenizer = AutoTokenizer.from_pretrained(GEMMA_3_4B_IT)

# Set the pad token to the EOS token for consistent sequence padding
if gemma_3_4b_it_tokenizer.pad_token is None:
    gemma_3_4b_it_tokenizer.pad_token = gemma_3_4b_it_tokenizer.eos_token

# Verify the model's chat template which dictates the required input format for tuning
print("\n--- Model's Chat Template ---")
print(gemma_3_4b_it_tokenizer.chat_template)
print("-----------------------------\n")

In [ ]:
# --- Attach NEW LoRA Adapter for Instruction-Tuning ---
# Define the LoRA configuration parameters
lora_config = LoraConfig(
    lora_alpha=32, # Scaling factor for LoRA weights
    lora_dropout=0.05, # Dropout rate for LoRA layers
    r=16, # Rank of the update matrices (lower rank = fewer trainable parameters)
    bias="none", # Do not fine-tune bias parameters
    # Target all linear layers (QKV, MLP projections) critical for effective LoRA
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM", # Specify the task as Causal Language Modeling
)

# Apply the LoRA adapter to the base model
gemma_3_4b_it_model = get_peft_model(gemma_3_4b_it_model, lora_config)

print("New LoRA adapter attached. Trainable parameters:")
# Display the proportion of trainable vs. total parameters (usually <1%)
gemma_3_4b_it_model.print_trainable_parameters()

In [ ]:
# --- Load and Prepare Content and Title Dataset for Instruction-Tuning ---
# Define the fixed system prompt used for fine-tuning the model's behavior
SYSTEM_PROMPT = """You are an expert-level, highly experienced Persian news editor. You have a deep understanding of journalistic standards, factual reporting, and the nuances of the Persian language.

Your task is to generate a single, high-quality, informative headline (title) for a given news report. You will be provided with the full text of the news content.

## **Instructions and Constraints:**

* **Language**: The generated title **MUST** be written primarily in the Persian language. English words should only be used if they are specific proper nouns (such as organization names, brands, or technical terms) **and** they appear in the original news content provided.
* **Length**: The title must be approximately **15 words** long.
* **Format**: The title **MUST** be a **declarative sentence**. It should state a fact or summarize the main point directly. It must **not** be phrased as a question.
* **Focus**: The primary goal is to be **informative and accurate**. The title must be 100% relevant to the provided news content and precisely summarize its core subject or most significant finding.

Analyze the provided news content, identify its most critical information, and then craft a single headline that meets all of the above criteria.

## **Output Format:**

Respond **only** with the generated Persian title. Do not add any commentary, explanations, or introductory text.

News report:
"""

chat_data = []
# Iterate through the training data to structure it into the model's chat message format
for index, row in train_data.iterrows():
    # Sanitize inputs
    content = str(row['content']).strip()
    title = str(row['title']).strip()

    if content and title:
        chat_data.append({
            # The instruction tuning format: System -> User (Content) -> Assistant (Target Title)
            "messages": [
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": content},
                {"role": "assistant", "content": title}
            ]
        })

print(f"Loaded {len(chat_data)} filtered Content and Title pairs for instruction-tuning.")

# Convert the list of messages into a Hugging Face Dataset object
dataset = Dataset.from_list(chat_data)
# Shuffle the dataset for better training robustness
dataset = dataset.shuffle(seed=42)

# Display a sample of the structured data
print("\n--- Example Data Sample (Unformatted Messages) ---")
print(dataset[0]['messages'][0])
print(dataset[0]['messages'][1])
print(dataset[0]['messages'][2])
print("---------------------------\n")

In [13]:
# --- Tokenization and Chat Template Application ---
def formatting_prompts_func(examples):
    """Converts the list of chat messages into a single string formatted according to the tokenizer's chat template."""
    texts = []
    for messages in examples["messages"]:
        # Apply the chat template to convert messages to the proper model input string
        text = gemma_3_4b_it_tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
        # Remove the leading <bos> token since the trainer will add it later during tokenization
        texts.append(text.removeprefix(gemma_3_4b_it_tokenizer.bos_token))
    return {"text": texts}

# Apply the formatting function to the entire dataset in a batched manner
dataset = dataset.map(formatting_prompts_func, batched=True)

# Inspect the final formatted data string
print("Formatted example (first 500 characters):")
print(dataset[0]["text"][:500] + "..." if len(dataset[0]["text"]) > 500 else dataset[0]["text"])
print("\n" + "="*50)

# Optional: Check multiple examples to ensure formatting is correct
print("Sample formatted outputs:")
for i in range(min(2, len(dataset))):  # Show first 2 examples
    print(f"\nExample {i+1}:")
    print(dataset[i]["text"])

# Final data verification
print(f"Loaded {len(dataset)} examples")

Map:   0%|          | 0/1044 [00:00<?, ? examples/s]

Formatted example (first 500 characters):
<start_of_turn>user
You are an expert-level, highly experienced Persian news editor. You have a deep understanding of journalistic standards, factual reporting, and the nuances of the Persian language.

Your task is to generate a single, high-quality, informative headline (title) for a given news report. You will be provided with the full text of the news content.

## **Instructions and Constraints:**

* **Language**: The generated title **MUST** be written primarily in the Persian language. Eng...

Sample formatted outputs:

Example 1:
<start_of_turn>user
You are an expert-level, highly experienced Persian news editor. You have a deep understanding of journalistic standards, factual reporting, and the nuances of the Persian language.

Your task is to generate a single, high-quality, informative headline (title) for a given news report. You will be provided with the full text of the news content.

## **Instructions and Constraints:**

* **Langu

In [ ]:
# --- Configure Trainer (TRL SFTTrainer) ---
print("Configuring SFTTrainer...")

# Define the training arguments using SFTConfig for Supervised Fine-Tuning
training_args = SFTConfig(
    output_dir=ADAPTER_OUTPUT_DIR,
    num_train_epochs=4, # Number of passes over the training data
    per_device_train_batch_size=2, # Batch size per GPU
    gradient_accumulation_steps=4, # Effectively increases batch size to 8
    optim="adamw_torch", # Optimizer choice
    logging_steps=5,
    save_strategy="epoch", # Save checkpoint after each epoch
    save_total_limit=10, # Limit the number of saved checkpoints
    learning_rate=1e-4, # Fine-tuning learning rate
    weight_decay=0.01,
    max_grad_norm=0.3,
    warmup_ratio=0.03, # Small warmup to stabilize training
    lr_scheduler_type="cosine", # Cosine schedule is generally robust
    gradient_checkpointing=True, # Saves memory at the cost of slight speed reduction
    gradient_checkpointing_kwargs={"use_reentrant": False},
    report_to="none", # Disable reporting to external platforms (e.g., W&B)
    dataset_text_field = "text", # Point the trainer to the formatted text column
)

# Initialize the SFTTrainer
trainer = SFTTrainer(
    model=gemma_3_4b_it_model,
    # The tokenizer must be passed for processing and padding
    tokenizer=gemma_3_4b_it_tokenizer,
    train_dataset=dataset,
    args=training_args,
)

# Crucial step: Mask the loss calculation so that only the model's generated response (assistant role)
# is used for backpropagation, not the system or user instructions.
trainer = train_on_responses_only(
    trainer,
    instruction_part = gemma_3_4b_it_tokenizer.apply_chat_template([{"role": "user", "content": ""}], tokenize=False, add_generation_prompt=False).replace("<bos>", "").replace("user\n", ""),
    response_part = gemma_3_4b_it_tokenizer.apply_chat_template([{"role": "assistant", "content": ""}], tokenize=False, add_generation_prompt=False).replace("<bos>", "").replace("assistant\n", ""),
)
# Note: The `train_on_responses_only` implementation is complex and often model-specific.
# The original code's approach using hardcoded markers might be safer if the template is guaranteed.
# Reverting to the simpler, likely intended approach, assuming the template is consistent:
# trainer = train_on_responses_only(
#     trainer,
#     instruction_part = "<start_of_turn>user\n",
#     response_part = "<start_of_turn>model\n",
# )

In [ ]:
# --- Start Training ---
print("Starting instruction fine-tuning...")
# Begin the training process
trainer.train()
print("Training finished.")

# --- Save the final adapter ---
# Create a dedicated path for the final LoRA adapter weights
final_adapter_path = os.path.join(ADAPTER_OUTPUT_DIR, "final_checkpoint")
# Save only the LoRA weights, not the entire base model
trainer.save_model(final_adapter_path)
print(f"Instruction-tuning adapter saved to: {final_adapter_path}")

## Step 3.2: Fine-Tuning Gemma 3 – 4B – IT using LoRA


In [6]:
# --- Configuration ---
# Llama 3.1 - 8B instruction-tuned model identifier
LLAMA_31_8B_IT = "meta-llama/Meta-Llama-3.1-8B-Instruct"

# Defined output directory for saving the LoRA adapter checkpoints
ADAPTER_OUTPUT_DIR = "/content/drive/MyDrive/Colab Notebooks/Radin/Llama_Output"

In [7]:
# --- Load The Base Model ---
llama_model = AutoModelForCausalLM.from_pretrained(
    LLAMA_31_8B_IT,
    torch_dtype=torch.bfloat16, # Use bfloat16 for speed and VRAM efficiency
    device_map="auto", # Automatically map model layers to available devices
    low_cpu_mem_usage=True, # Optimized loading
)

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

In [8]:
## --- Load The Tokenizer ---
llama_tokenizer = AutoTokenizer.from_pretrained(LLAMA_31_8B_IT)

# Ensure pad token is set to the EOS token for batching readiness
if llama_tokenizer.pad_token is None:
    llama_tokenizer.pad_token = llama_tokenizer.eos_token
    llama_tokenizer.pad_token_id = llama_tokenizer.eos_token_id

print("\n--- Model's Chat Template ---")
# Display the required instruction format for Llama 3.1
print(llama_tokenizer.chat_template)
print("-----------------------------\n")

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]


--- Model's Chat Template ---
{{- bos_token }}
{%- if custom_tools is defined %}
    {%- set tools = custom_tools %}
{%- endif %}
{%- if not tools_in_user_message is defined %}
    {%- set tools_in_user_message = true %}
{%- endif %}
{%- if not date_string is defined %}
    {%- set date_string = "26 Jul 2024" %}
{%- endif %}
{%- if not tools is defined %}
    {%- set tools = none %}
{%- endif %}

{#- This block extracts the system message, so we can slot it into the right place. #}
{%- if messages[0]['role'] == 'system' %}
    {%- set system_message = messages[0]['content']|trim %}
    {%- set messages = messages[1:] %}
{%- else %}
    {%- set system_message = "" %}
{%- endif %}

{#- System message + builtin tools #}
{{- "<|start_header_id|>system<|end_header_id|>\n\n" }}
{%- if builtin_tools is defined or tools is not none %}
    {{- "Environment: ipython\n" }}
{%- endif %}
{%- if builtin_tools is defined %}
    {{- "Tools: " + builtin_tools | reject('equalto', 'code_interpreter') | 

In [ ]:
## --- Attach NEW LoRA Adapter ---
# Configure LoRA parameters (consistent with Gemma for comparison)
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    # Target projection layers across the attention and feed-forward blocks
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
)

# Apply the LoRA adapter to the Llama base model
llama_model = get_peft_model(llama_model, lora_config)

print("New LoRA adapter attached. Trainable parameters:")
# Print the ratio of trainable parameters to total parameters
llama_model.print_trainable_parameters()

## **Load and Prepare Content and Title Dataset**

In [ ]:
## --- Load and Prepare Content and Title Dataset ---
# Define the fixed system prompt used for instruction-tuning
SYSTEM_PROMPT = """You are an expert-level, highly experienced Persian news editor. You have a deep understanding of journalistic standards, factual reporting, and the nuances of the Persian language.

Your task is to generate a single, high-quality, informative headline (title) for a given news report. You will be provided with the full text of the news content.

## **Instructions and Constraints:**

* **Language**: The generated title **MUST** be written primarily in the Persian language. English words should only be used if they are specific proper nouns (such as organization names, brands, or technical terms) **and** they appear in the original news content provided.
* **Length**: The title must be approximately **15 words** long.
* **Format**: The title **MUST** be a **declarative sentence**. It should state a fact or summarize the main point directly. It must **not** be phrased as a question.
* **Focus**: The primary goal is to be **informative and accurate**. The title must be 100% relevant to the provided news content and precisely summarize its core subject or most significant finding.

Analyze the provided news content, identify its most critical information, and then craft a single headline that meets all of the above criteria.

## **Output Format:**

Respond **only** with the generated Persian title. Do not add any commentary, explanations, or introductory text.

News report:
"""

chat_data = []
# Iterate through the training data to structure it into the model's message format
for index, row in train_data.iterrows():
    content = str(row['content']).strip()
    title = str(row['title']).strip()

    if content and title:
        chat_data.append({
            "messages": [
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": content},
                {"role": "assistant", "content": title} # Target output
            ]
        })

print(f"Loaded {len(chat_data)} filtered Content and Title pairs for instruction-tuning.")

# Convert to Hugging Face Dataset
dataset = Dataset.from_list(chat_data)
# Shuffle the dataset for robust training
dataset = dataset.shuffle(seed=42)

print("\n--- Example Data Sample (Raw Messages) ---")
for msg in dataset[0]['messages']:
    print(msg)
print("---------------------------\n")

In [11]:
## --- Format Dataset Using Chat Template ---
def formatting_prompts_func(examples):
    """Applies the Llama tokenizer's chat template to structure the messages into the final training string."""
    texts = []
    for messages in examples["messages"]:
        # Apply the Llama 3.1 template without adding the initial generation prompt
        text = llama_tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
        texts.append(text)
    return {"text": texts}

# Apply the function to the entire dataset
dataset = dataset.map(formatting_prompts_func, batched=True)

# Inspect formatted example (first 500 characters)
print("Formatted example (first 500 characters):")
print(dataset[0]["text"][:500] + "..." if len(dataset[0]["text"]) > 500 else dataset[0]["text"])
print("\n" + "="*50)

# Display a full sample of the formatted output
print("Sample formatted outputs:")
for i in range(min(2, len(dataset))):
    print(f"\nExample {i+1}:")
    print(dataset[i]["text"])

print(f"Loaded {len(dataset)} examples")

Map:   0%|          | 0/1044 [00:00<?, ? examples/s]

Formatted example (first 500 characters):
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are an expert-level, highly experienced Persian news editor. You have a deep understanding of journalistic standards, factual reporting, and the nuances of the Persian language.

Your task is to generate a single, high-quality, informative headline (title) for a given news report. You will be provided with the full text of the news content.

## **Instructions and Constr...

Sample formatted outputs:

Example 1:
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are an expert-level, highly experienced Persian news editor. You have a deep understanding of journalistic standards, factual reporting, and the nuances of the Persian language.

Your task is to generate a single, high-quality, informative headline (title) for a given news report. You w

In [12]:
## --- Configure Trainer (SFTTrainer) ---
training_args = SFTConfig(
    output_dir=ADAPTER_OUTPUT_DIR,
    num_train_epochs=4, # Consistent number of epochs
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    optim="adamw_torch",
    logging_steps=5,
    save_strategy="epoch",
    save_total_limit=10,
    learning_rate=1e-4,
    weight_decay=0.01,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="cosine",
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={"use_reentrant": False},
    report_to="none",
    dataset_text_field="text",
)

trainer = SFTTrainer(
    model=llama_model,
    # Pass the tokenizer to the processing_class argument
    tokenizer=llama_tokenizer,
    train_dataset=dataset,
    args=training_args,
)

# Apply loss masking using the Llama 3.1 specific header tokens
trainer = train_on_responses_only(
    trainer,
    instruction_part="<|start_header_id|>user<|end_header_id|>\n\n",
    response_part="<|start_header_id|>assistant<|end_header_id|>\n\n",
)

Unsloth: Tokenizing ["text"] (num_proc=16):   0%|          | 0/1044 [00:00<?, ? examples/s]

In [ ]:
## --- Start Training ---
print("Starting instruction fine-tuning...")
# Execute the training loop
trainer.train()
print("Training finished.")

# Save final adapter weights to the specified output directory
final_adapter_path = os.path.join(ADAPTER_OUTPUT_DIR, "final_checkpoint")
trainer.save_model(final_adapter_path)
print(f"LoRA adapter saved to: {final_adapter_path}")

## Step 4: Inference and Quantitative Evaluation

This phase executes the final, quantitative evaluation of both LLMs across all five test configurations: the four **prompt-based strategies** (Simple, Zero-Shot, Few-Shot, Refined) and the **LoRA Fine-Tuned** model. The goal is to generate predictions for the entire hold-out test set, which will then be evaluated using numerical metrics.

This is structured into three main steps per model:
1.  **Stage 1: Base Model Inference:** Generate predictions using the instruction-tuned model with all four prompt templates.
2.  **Stage 2: Fine-Tuned Model Inference:** Load the saved LoRA adapter and generate predictions using the dedicated fine-tuning system prompt.
3.  **Stage 3: Result Consolidation:** Save all generated titles alongside the ground truth for later metric calculation.

In [7]:
# --- Prompt Definitions (Re-defined for modularity) ---
# The prompt templates defined in Step 2 are needed for inference on the base models.

# 1. Simple, Minimalist Prompt
simple_prompt = """
Generate a high-quality headline for the following news report, responding only with the Persian title and no other text.

News report:
"""
# 2. Detailed, Persona-Based Prompt (Zero-Shot)
zero_shot_prompt = """You are an expert-level, highly experienced Persian news editor. You have a deep understanding of journalistic standards, factual reporting, and the nuances of the Persian language.

Your task is to generate a single, high-quality, informative headline (title) for a given news report. You will be provided with the full text of the news content.

## **Instructions and Constraints:**

* **Language**: The generated title **MUST** be written primarily in the Persian language. English words should only be used if they are specific proper nouns (such as organization names, brands, or technical terms) **and** they appear in the original news content provided.
* **Length**: The title must be approximately **15 words** long.
* **Format**: The title **MUST** be a **declarative sentence**. It should state a fact or summarize the main point directly. It must **not** be phrased as a question.
* **Focus**: The primary goal is to be **informative and accurate**. The title must be 100% relevant to the provided news content and precisely summarize its core subject or most significant finding.

Analyze the provided news content, identify its most critical information, and then craft a single headline that meets all of the above criteria.

## **Output Format:**

Respond **only** with the generated Persian title. Do not add any commentary, explanations, or introductory text.

News report:
"""
# 3. Few-Shot Prompt (with examples)
few_shot_prompt = """You are an expert-level, highly experienced Persian news editor. You have a deep understanding of journalistic standards, factual reporting, and the nuances of the Persian language.

Your task is to generate a single, high-quality, informative headline (title) for a given news report. You will be provided with the full text of the news content.

## **Instructions and Constraints:**

* **Language**: The generated title **MUST** be written primarily in the Persian language. English words should only be used if they are specific proper nouns (such as organization names, brands, or technical terms) **and** they appear in the original news content provided.
* **Length**: The title must be approximately **15 words** long.
* **Format**: The title **MUST** be a **declarative sentence**. It should state a fact or summarize the main point directly. It must **not** be phrased as a question.
* **Focus**: The primary goal is to be **informative and accurate**. The title must be 100% relevant to the provided news content and precisely summarize its core subject or most significant finding.

Analyze the provided news content, identify its most critical information, and then craft a single headline that meets all of the above criteria.

## **Examples:**

Here are five examples of correct `Content` to `Title` generation:

---
**Example 1**

**Content:**
شرح ادعا کاربری در ایکس ادعا کرده است که بیش از هزار کارگر
طی شش ماه در حوادث ضمن کار جان خود را از دست داده‌اند. نتیجه
بررسی براساس آمار اختصاصی سازمان پزشکی قانونی : در شش ماهه
نخست سال جاری ۱۰۷۷ نفر در حوادث کار جان خود را از دست دادند.
این رقم در مقایسه با مدت مشابه سال قبل با آمار فوتی ۹۳۱ نفر،
۱۵/۷ درصد افزایش داشته است. آمار اختصاصی سازمان پزشکی قانونی
در سایت باشگاه خبرنگاران جوان براساس سالنامه آماری وزارت کار
در سال ۱۴۰۱ : تعداد متوفیان حوادث ناشی از کار ثبت شده در
مراکز پزشکی قانونی در سال ۱۴۰۱ ، ۱۹۰۰ نفر بوده است. سالنامه
آماری وزارت کار در سال ۱۴۰۱

**Title:**
بیش از هزار کارگر طی شش ماه در حوادث ضمن کار کشته شده‌اند

---
**Example 2**

**Content:**
حسن روحانی در آیین افتتاح طرح‌های ملی وزارت جهاد کشاورزی، در
هشتاد و دومین پویش تدبیر و امید برای جهش تولید گفت: « مردم
خوزستان علی‌رغم اینکه توصیه ما این بود که آنجا برنج‌کاری
نشود، اما سال‌هاست برنج‌کاری هم انجام می‌دهند که خودِ آن هم
مزید بر علت شده و این‌ها همه دست به دست هم داده و مردم عزیز
در سختی قرار گرفتند.» صفحه شبکه منوتو در اینستاگرام به نقل
از حسن روحانی نوشته است: «مردم خوزستان علی‌Rateرغم اینکه توصیه
ما این بود که در آنجا برنج‌کاری نشود، اما سال‌هاست برنج‌کاری
هم انجام می‌ده دهند که خود آن هم مزید بر علت در بحران کم‌آبی
شده است.» به گزارش سایت رسمی پایگاه اطلاع رسانی ریاست جمهوری
به تاریخ ۲۸ تیرماه، حسن روحانی در آیین افتتاح طرح‌های ملی
وزارت جهاد کشاورزی در هشتاد و دومین پویش تدبیر و امید برای
جهش تولید گفت: «مردم خوزستان علی‌رغم اینکه توصیه ما این بود
که آنجا برنج‌کاری نشود؛ اما سال‌هاست برنج‌کاری هم انجام
می‌دهند که خودِ آن هم مزید بر علت شده و این‌ها همه دست به
دست هم داده و مردم عزیز در سختی قرار گرفتند.» سایت رسمی
پایگاه اطلاع رسانی ریاست جمهوری

**Title:**
حسن روحانی گفته‌است: «برنج‌کاری مردم خوزستان از دلایل بحران
کم‌آبی است.»

---
**Example 3**

**Content:**
شرح ادعا کاربری در شبکه اجتماعی ایکس ادعا کرده‌است که فقط در
۸ ماه، ۴ هزار و ۵۰۰ پزشک و پرستار از ایران مهاجرت کرده‌اند.
نتیجه بررسی این آمار، براساس گزارش روزنامه ایران از شمار
گواهی‌های صادر شده گود استندینگ (Good standing) صادر شده
توسط سازمان نظام پزشکی است. گواهی گود استندینگ یکی از مدارکی
است که اعضای کادر درمان باید برای مهاجTرت و فعالیت حرفه‌ای در
کشورهای خارجی ارائه دهند؛ این گواهی حسن سابقه افراد را مشخص
می‌کند و یعنی این افراد صلاحیت لازم را برای فعالیت در کشور
مقصد را دارند. بر اساس این گزارش، در 8 ماه ابتدایی امسال،
4500 نفر گواهی گود استندینگ دریافت کرده‌اند، اما صرف دریافت
این گواهی به معنای مهاجTرت قطعی نیست. بنابراین، اگرچه دریافت
گواهی گوداستندینگ برای اقدام در راستای مهاجرت است؛ اما به
این معنا که هر کس این گواهی را دریافت کرده، سپس مهاجرت
نموده، نیست. پس ادعای این کاربر گمراه‌کننده ارزیابی می‌شود.

**Title:**
در 8 ماه ابتدایی سال جاری، 4500 پزشک و پرستار مهاجرت
کرده‌اند

---
**Example 4**

**Content:**
گرفتن حق اشتراک و پخش آگهی قبل از انتشار محتوا توسط
پلتفرم‌های اینترنتی در هیچ جای دنیا مرسوم نیست و به زودی
برای آن دستورالعمل قانونی نوشته و پس از مصوب‌ شدن، به آن‌ها
ابلاغ می‌شود. در حال حاضر، قانونی که پلتفرم‌ها را از این عمل
منع کند وجود ندارد. مصطفی قاسمیان، روزنامه‌نگار سینما و
تلویزیون، در توییتر ادعا کرده است: «با وجودی که ۵ ماه پیش
ساترا صراحتا اعلام کرد پلتفرم‌هایی که هزینه اشتراک می‌گیرند،
حق نمایش آگهی پیش از پخش محتوا ندارند، اما فیلیمو و نماوا
همچنان این عمل غیرقانونی را انجام می‌دهند.» توییت قاسمیان
درباره پخش آگهی فیلیمو و نماوا علی سعد، معاون کاربران و
تنظیم مقررات اجتماعی ساترا (سازمان تنظیم مقررات صوت و تصویر
فراگیر در فضای مجازی) در تماس تلفنی با فکت‌یار گفت : «ما
فروردین ماه اعلام کردیم... در حال حاضر
این دو پلتفرم کار غیر‌قانونی انجام نمی‌دهند چون هنوز قانون
آن مصوب و ابلاغ نشده است.»

**Title:**
با وجود اعلام ساترا مبنی بر اینکه پلتفرم‌هایی که هزینه
اشتراک می‌گیرند، حق نمایش آگهی پیش از پخش محتوا را ندارند،
فیلیمو و نماوا با انتشار آگهی قبل از نمایش محتوا عمل
غیرقانونی انجام می‌دهند.

---
**Example 5**

**Content:**
شرح ادعا عبدالرضا داوری با انتشار فیلمی از مردی که برای
دریافت هزینه اشتراک تلویزیون BBC به خانه یکی از شهروندان این
کشور رفته‌است، گفت: «در حالیکه مردم بریتانیا مجبور هستند
برای شبکه BBC حق اشتراک بپردازند، شبکه BBC چه منافعی در
ایران دارد که برنامه‌های بخش فارسی آن، بصورت رایگان و ۲۴
ساعته برای ملت ایران پخش می‌شود؟». نتیجه بررسی براساس
آیین‌نامه‌ای که در وبسایت دولت بریتانیا برای دریافت مجوز
دیدن تلویزیون وجود دارد ؛ شهروندان برای دیدن برنامه‌های زنده
تلویزیون، به صورت کلی و برای همه شبکه‌ها (اعم از BBC)، باید
۱۶۹.۵پوند پرداخت کنند. دیدن بی بی سی فارسی، به صورت آنلاین
یا با استفاده از آنتن‌های ماهواره، برای مخاطبان رایگان است.
بنابراین، ادعای عبدالرضا داوری در مورد اینکه مردم بریتانیا
باید برای دیدن BBC اشتراک تهیه کنند اما این شبکه به طور
رایگان برای مردم ایران به طور 24 ساعته برنامه‌سازی می‌کند،
درست ارزیابی می‌شود.

**Title:**
مردم بریتانیا باید برای دیدن BBC اشتراک بخرند؛ در حالی که
دیدن BBC فارسی برای مردم ایران رایگان است

---

## **Output Format:**

Respond **only** with the generated Persian title. Do not add any commentary, explanations, or introductory text.

News report:
"""
# 4. Highly Refined Prompt (with explicit claim logic and examples)
refined_prompt = """
You are an expert-level, highly experienced Persian news editor. Your task is to generate a single, high-quality, informative headline (title) based on a given news report.

The content you will receive is a fact-check, which contains two main parts:
1.  A **"شرح ادعا" (Claim Description)**
2.  A **"نتیجه بررسی" (Verification / Result)**

## **Core Logic: Handling Claims vs. Verifications**

Your primary task is to identify the most newsworthy part of the article to summarize as a headline.

* The content you receive is a fact-check. In this format, the **"Claim" (ادعا)** is almost always the main subject and the most newsworthy part.
* **Crucial Instruction:** In many cases, even when the **"Verification" (نتیجه بررسی)** finds that the "Claim" is **incorrect (نادرست)** or **misleading (گمراه‌کننده)**, the claim is *still* the main subject of the article.
* **In these specific cases, your headline MUST be a declarative statement of that incorrect or semi-correct claim.**
* The examples (especially Example 3 and Example 4) clearly demonstrate this rule. In both, the verification finds the claim to be false or misleading, but the correct `Title` is still a statement of the `Claim`.

## **Instructions and Constraints:**

* **Language**: The generated title **MUST** be written primarily in the Persian language. English words should only be used if they are specific proper nouns (such as organization names, brands, or technical terms) **and** they appear in the original "شرح ادعا".
* **Length**: The title must be approximately **15 words** long.
* **Format**: The title **MUST** be a **declarative sentence**. It must **not** be phrased as a question.

Analyze the provided news content, identify the core claim, and then craft a single headline that meets all of the above criteria based on the Core Logic.

## **Examples:**

Here are five examples showing how the `Content` (which includes a claim and a verification) is used to create a `Title` (which is *only* the claim).

---
**Example 1**

**Content:**
شرح ادعا کاربری در ایکس ادعا کرده است که بیش از هزار کارگر
طی شش ماه در حوادث ضمن کار جان خود را از دست داده‌اند. نتیجه
بررسی براساس آمار اختصاصی سازمان پزشکی قانونی : در شش ماهه
نخست سال جاری ۱۰۷۷ نفر در حوادث کار جان خود را از دست دادند.
این رقم در مقایسه با مدت مشابه سال قبل با آمار فوتی ۹۳۱ نفر،
۱۵/۷ درصد افزایش داشته است. آمار اختصاصی سازمان پزشکی قانونی
در سایت باشگاه خبرنگاران جوان براساس سالنامه آماری وزارت کار
در سال ۱۴۰۱ : تعداد متوفیان حوادث ناشی از کار ثبت شده در
مراکز پزشکی قانونی در سال ۱۴۰۱ ، ۱۹۰۰ نفر بوده است. سالنامه
آماری وزارت کار در سال ۱۴۰۱

**Title:**
بیش از هزار کارگر طی شش ماه در حوادث ضمن کار کشته شده‌اند

---
**Example 2**

**Content:**
حسن روحانی در آیین افتتاح طرح‌های ملی وزارت جهاد کشاورزی، در
هشتاد و دومین پویش تدبیر و امید برای جهش تولید گفت: « مردم
خوزستان علی‌رغم اینکه توصیه ما این بود که آنجا برنج‌کاری
نشود، اما سال‌هاست برنج‌کاری هم انجام می‌دهند که خودِ آن هم
مزید بر علت شده و این‌ها همه دست به دست هم داده و مردم عزیز
در سختی قرار گرفتند.» صفحه شبکه منوتو در اینستاگرام به نقل
از حسن روحانی نوشته است: «مردم خوزستان علی‌Rateرغم اینکه توصیه
ما این بود که در آنجا برنج‌کاری نشود، اما سال‌هاست برنج‌کاری
هم انجام می‌ده دهند که خود آن هم مزید بر علت در بحران کم‌آبی
شده است.» به گزارش سایت رسمی پایگاه اطلاع رسانی ریاست جمهوری
به تاریخ ۲۸ تیرماه، حسن روحانی در آیین افتتاح طرح‌های ملی
وزارت جهاد کشاورزی در هشتاد و دومین پویش تدبیر و امید برای
جهش تولید گفت: «مردم خوزستان علی‌رغم اینکه توصیه ما این بود
که آنجا برنج‌کاری نشود؛ اما سال‌هاست برنج‌کاری هم انجام
می‌دهند که خودِ آن هم مزید بر علت شده و این‌ها همه دست به
دست هم داده و مردم عزیز در سختی قرار گرفتند.» سایت رسمی
پایگاه اطلاع رسانی ریاست جمهوری

**Title:**
حسن روحانی گفته‌است: «برنج‌کاری مردم خوزستان از دلایل بحران
کم‌آبی است.»

---
**Example 3**

**Content:**
شرح ادعا کاربری در شبکه اجتماعی ایکس ادعا کرده‌است که فقط در
۸ ماه، ۴ هزار و ۵۰۰ پزشک و پرستار از ایران مهاجرت کرده‌اند.
نتیجه بررسی این آمار، براساس گزارش روزنامه ایران از شمار
گواهی‌های صادر شده گود استندینگ (Good standing) صادر شده
توسط سازمان نظام پزشکی است. گواهی گود استندینگ یکی از مدارکی
است که اعضای کادر درمان باید برای مهاجTرت و فعالیت حرفه‌ای در
کشورهای خارجی ارائه دهند؛ این گواهی حسن سابقه افراد را مشخص
می‌کند و یعنی این افراد صلاحیت لازم را برای فعالیت در کشور
مقصد را دارند. بر اساس این گزارش، در 8 ماه ابتدایی امسال،
4500 نفر گواهی گود استندینگ دریافت کرده‌اند، اما صرف دریافت
این گواهی به معنای مهاجTرت قطعی نیست. بنابراین، اگرچه دریافت
گواهی گوداستندینگ برای اقدام در راستای مهاجرت است؛ اما به
این معنا که هر کس این گواهی را دریافت کرده، سپس مهاجرت
نموده، نیست. پس ادعای این کاربر گمراه‌کننده ارزیابی می‌شود.

**Title:**
در 8 ماه ابتدایی سال جاری، 4500 پزشک و پرستار مهاجرت
کرده‌اند

---
**Example 4**

**Content:**
گرفتن حق اشتراک و پخش آگهی قبل از انتشار محتوا توسط
پلتفرم‌های اینترنتی در هیچ جای دنیا مرسوم نیست و به زودی
برای آن دستورالعمل قانونی نوشته و پس از مصوب‌ شدن، به آن‌ها
ابلاغ می‌شود. در حال حاضر، قانونی که پلتفرم‌ها را از این عمل
منع کند وجود ندارد. مصطفی قاسمیان، روزنامه‌نگار سینما و
تلویزیون، در توییتر ادعا کرده است: «با وجودی که ۵ ماه پیش
ساترا صراحتا اعلام کرد پلتفرم‌هایی که هزینه اشتراک می‌گیرند،
حق نمایش آگهی پیش از پخش محتوا ندارند، اما فیلیمو و نماوا
همچنان این عمل غیرقانونی را انجام می‌دهند.» توییت قاسمیان
درباره پخش آگهی فیلیمو و نماوا علی سعد، معاون کاربران و
تنظیم مقررات اجتماعی ساترا (سازمان تنظیم مقررات صوت و تصویر
فراگیر در فضای مجازی) در تماس تلفنی با فکت‌یار گفت : «ما
فروردین ماه اعلام کردیم... در حال حاضر
این دو پلتفرم کار غیر‌قانونی انجام نمی‌دهند چون هنوز قانون
آن مصوب و ابلاغ نشده است.»

**Title:**
با وجود اعلام ساترا مبنی بر اینکه پلتفرم‌هایی که هزینه
اشتراک می‌گیرند، حق نمایش آگهی پیش از پخش محتوا را ندارند،
فیلیمو و نماوا با انتشار آگهی قبل از نمایش محتوا عمل
غیرقانونی انجام می‌دهند.

---
**Example 5**

**Content:**
شرح ادعا عبدالرضا داوری با انتشار فیلمی از مردی که برای
دریافت هزینه اشتراک تلویزیون BBC به خانه یکی از شهروندان این
کشور رفته‌است، گفت: «در حالیکه مردم بریتانیا مجبور هستند
برای شبکه BBC حق اشتراک بپردازند، شبکه BBC چه منافعی در
ایران دارد که برنامه‌های بخش فارسی آن، بصورت رایگان و ۲۴
ساعته برای ملت ایران پخش می‌شود؟». نتیجه بررسی براساس
آیین‌نامه‌ای که در وبسایت دولت بریتانیا برای دریافت مجوز
دیدن تلویزیون وجود دارد ؛ شهروندان برای دیدن برنامه‌های زنده
تلویزیون، به صورت کلی و برای همه شبکه‌ها (اعم از BBC)، باید
۱۶۹.۵پوند پرداخت کنند. دیدن بی بی سی فارسی، به صورت آنلاین
یا با استفاده از آنتن‌های ماهواره، برای مخاطبان رایگان است.
بنابراین، ادعای عبدالرضا داوری در مورد اینکه مردم بریتانیا
باید برای دیدن BBC اشتراک تهیه کنند اما این شبکه به طور
رایگان برای مردم ایران به طور 24 ساعته برنامه‌سازی می‌کند،
درست ارزیابی می‌شود.

**Title:**
مردم بریتانیا باید برای دیدن BBC اشتراک بخرند؛ در حالی که
دیدن BBC فارسی برای مردم ایران رایگان است

---

## **Output Format:**

Respond **only** with the generated Persian title. Do not add any commentary, explanations, or introductory text.

News report:
"""
# 5. Dedicated System Prompt used during LoRA Fine-Tuning
fine_tuning_system_prompt = """You are an expert-level, highly experienced Persian news editor. You have a deep understanding of journalistic standards, factual reporting, and the nuances of the Persian language.

Your task is to generate a single, high-quality, informative headline (title) for a given news report. You will be provided with the full text of the news content.

## **Instructions and Constraints:**

* **Language**: The generated title **MUST** be written primarily in the Persian language. English words should only be used if they are specific proper nouns (such as organization names, brands, or technical terms) **and** they appear in the original news content provided.
* **Length**: The title must be approximately **15 words** long.
* **Format**: The title **MUST** be a **declarative sentence**. It should state a fact or summarize the main point directly. It must **not** be phrased as a question.
* **Focus**: The primary goal is to be **informative and accurate**. The title must be 100% relevant to the provided news content and precisely summarize its core subject or most significant finding.

Analyze the provided news content, identify its most critical information, and then craft a single headline that meets all of the above criteria.

## **Output Format:**

Respond **only** with the generated Persian title. Do not add any commentary, explanations, or introductory text.

News report:
"""

### Step 4.1: Inference on Gemma 3 – 4B – IT

This step executes the quantitative inference for the **Gemma 3 – 4B – IT** model. We perform inference in two stages—first with the base model and various prompts, then with the LoRA-adapted model—to efficiently manage VRAM, clearing memory between stages. The generated titles are cleaned and collected for final metric evaluation.

In [ ]:
import pandas as pd
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
from tqdm.auto import tqdm
import gc
import re
import os

# --- 1. Define Helper Functions ---
def clean_output(text: str) -> str:
    """Removes common model artifacts and leading/trailing whitespace from generated text."""
    if not isinstance(text, str):
        return ""
    # Remove potential "Title:", "Headline:", or similar prefixes, case-insensitive
    text = re.sub(r"^(title|headline)\s*:\s*", "", text, flags=re.IGNORECASE)
    return text.strip()

def format_for_gemma(system_prompt_text, query_text):
    """Formats a system prompt and query into the Gemma chat_template format for the BASE model pipeline."""
    # Note: This is an approximation for the chat_template using pipeline's messages format
    messages = [
        {
            "role": "system",
            "content": [{"type": "text", "text": system_prompt_text}]
        },
        {
            "role": "user",
            "content": [{"type": "text", "text": query_text}]
        }
    ]
    return messages

# --- 2. Define Model/Adapter Paths ---
GEMMA_3_4B_IT = "google/gemma-3-4b-it"
ADAPTER_OUTPUT_DIR_GEMMA = "/content/drive/MyDrive/Colab Notebooks/Radin/Gemma_Output"

# This list will hold our dictionaries of results for Gemma
gemma_results_list = []
print(f"Initializing results collection for {len(test_data)} test items.")

# -----------------------------------------------------------------
# STAGE 1: INFERENCE WITH BASE MODEL (4 Prompts)
# -----------------------------------------------------------------
print("\n--- STAGE 1: Loading BASE Gemma model for prompt testing ---")
base_gemma_model = AutoModelForCausalLM.from_pretrained(
    GEMMA_3_4B_IT,
    dtype=torch.bfloat16,
    device_map="auto",
    low_cpu_mem_usage=True,
)
base_gemma_tokenizer = AutoTokenizer.from_pretrained(GEMMA_3_4B_IT)
if base_gemma_tokenizer.pad_token is None:
    base_gemma_tokenizer.pad_token = base_gemma_tokenizer.eos_token

base_gemma_pipe = pipeline(
    task="text-generation",
    model=base_gemma_model,
    tokenizer=base_gemma_tokenizer,
    dtype=torch.bfloat16
)
# Generation parameters for prompt-based inference
gemma_3_4b_it_pipe_kwargs = {
    "max_new_tokens": 128,
    "return_full_text": False,
    "do_sample": True,
    "temperature": 0.7,
    "top_p": 0.9,
    "pad_token_id": base_gemma_tokenizer.eos_token_id
}
print("Base Gemma model loaded. Running 4 prompt configurations on test set...")

# --- Run Inference on Test Set (Base Model) ---
for index, row in tqdm(test_data.iterrows(), total=len(test_data), desc="Processing Gemma (Base)"):
    content = str(row['content']).strip()
    actual_title = str(row['title']).strip()

    if not content:
        continue

    # Generate title for each of the four prompts
    formatted_simple = format_for_gemma(simple_prompt, content)
    simple_gen = base_gemma_pipe(formatted_simple, **gemma_3_4b_it_pipe_kwargs)[0]["generated_text"]

    formatted_zero_shot = format_for_gemma(zero_shot_prompt, content)
    zero_shot_gen = base_gemma_pipe(formatted_zero_shot, **gemma_3_4b_it_pipe_kwargs)[0]["generated_text"]

    formatted_few_shot = format_for_gemma(few_shot_prompt, content)
    few_shot_gen = base_gemma_pipe(formatted_few_shot, **gemma_3_4b_it_pipe_kwargs)[0]["generated_text"]

    formatted_refined = format_for_gemma(refined_prompt, content)
    refined_gen = base_gemma_pipe(formatted_refined, **gemma_3_4b_it_pipe_kwargs)[0]["generated_text"]

    # Store results, applying the cleanup function immediately
    gemma_results_list.append({
        'content': content,
        'actual_title': actual_title,
        'predicted_simple': clean_output(simple_gen),
        'predicted_zero_shot': clean_output(zero_shot_gen),
        'predicted_few_shot': clean_output(few_shot_gen),
        'predicted_refined': clean_output(refined_gen),
    })

# --- Cleanup Stage 1 ---
print("Base model inference complete. Clearing memory...")
del base_gemma_model, base_gemma_pipe, base_gemma_tokenizer
gc.collect()
torch.cuda.empty_cache()
print("Memory cleared.")

# -----------------------------------------------------------------
# STAGE 2: INFERENCE WITH FINE-TUNED MODEL (1 Prompt)
# -----------------------------------------------------------------
print("\n--- STAGE 2: Loading FINE-TUNED Gemma model and LoRA adapter ---")

# Load base model again to attach adapter
ft_gemma_base_model = AutoModelForCausalLM.from_pretrained(
    GEMMA_3_4B_IT,
    dtype=torch.bfloat16,
    device_map="auto",
    low_cpu_mem_usage=True,
)
# Attach the trained LoRA adapter weights
fine_tuned_gemma_model = PeftModel.from_pretrained(
    ft_gemma_base_model,
    os.path.join(ADAPTER_OUTPUT_DIR_GEMMA, "final_checkpoint")
)
ft_gemma_tokenizer = AutoTokenizer.from_pretrained(GEMMA_3_4B_IT)
if ft_gemma_tokenizer.pad_token is None:
    ft_gemma_tokenizer.pad_token = ft_gemma_tokenizer.eos_token

ft_gemma_pipe = pipeline(
    task="text-generation",
    model=fine_tuned_gemma_model,
    tokenizer=ft_gemma_tokenizer,
    dtype=torch.bfloat16
)
# Generation parameters for fine-tuned inference (often tighter limits)
ft_gemma_pipe_kwargs = {
    "max_new_tokens": 50, # Sufficient for title generation
    "return_full_text": False,
    "do_sample": True,
    "temperature": 0.7,
    "top_p": 0.9,
    "num_return_sequences": 1,
    "pad_token_id": ft_gemma_tokenizer.eos_token_id
}
print("Fine-tuned Gemma model loaded. Running fine-tuning prompt...")

# --- Run Inference on Test Set (Fine-Tuned Model) ---
# Iterate through the pre-populated results list to add the final column
for i in tqdm(range(len(gemma_results_list)), desc="Processing Gemma (FT)"):
    content = gemma_results_list[i]['content'] # Get content from our stored list

    # Format input using the tokenizer's chat template for the fine-tuned model
    messages_ft = [
        {"role": "system", "content": fine_tuning_system_prompt},
        {"role": "user", "content": content},
    ]
    formatted_input_ft = ft_gemma_tokenizer.apply_chat_template(
        messages_ft, tokenize=False, add_generation_prompt=True
    )
    # Generate the prediction
    ft_gen = ft_gemma_pipe(formatted_input_ft, **ft_gemma_pipe_kwargs)[0]["generated_text"]

    # Add the fine-tuned prediction to the existing dictionary
    gemma_results_list[i]['predicted_fine_tuned'] = clean_output(ft_gen)

print("Fine-tuned inference complete.")

# --- Cleanup Stage 2 ---
print("Cleaning up Gemma models from memory...")
del ft_gemma_base_model, fine_tuned_gemma_model, ft_gemma_pipe, ft_gemma_tokenizer
gc.collect()
torch.cuda.empty_cache()
print("Memory cleared.")

# -----------------------------------------------------------------
# STAGE 3: SAVE RESULTS
# -----------------------------------------------------------------
print("\n--- STAGE 3: Saving all results to CSV ---")
# Convert the list of results into a DataFrame and save
gemma_df = pd.DataFrame(gemma_results_list)
gemma_df.to_csv("gemma_results.csv", index=False, encoding='utf-8-sig')

print("\n" + "="*50)
print(f"✅ Gemma results saved to gemma_results.csv")
print("First 5 rows of Gemma results:")
print(gemma_df.head())
print("="*50 + "\n")
print("\nAll Gemma inference tasks are finished.")

### Step 4.2: Inference on Llama 3.1 – 8B – IT

This step executes the quantitative inference for the **Llama 3.1 – 8B – IT** model, following the same two-stage, memory-conscious procedure established for Gemma. This ensures all five configurations (four prompts and one LoRA FT) are benchmarked on the identical test set for the Llama model, and the results are consolidated and saved to a dedicated CSV file.

In [ ]:
import pandas as pd
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
from tqdm.auto import tqdm
import gc
import re
import os

# --- 1. Define Helper Functions (re-defined locally for modularity) ---
def clean_output(text: str) -> str:
    """Removes common model artifacts and leading/trailing whitespace."""
    if not isinstance(text, str): return ""
    # Remove potential "Title:" or "Headline:" prefixes
    text = re.sub(r"^(title|headline)\s*:\s*", "", text, flags=re.IGNORECASE)
    return text.strip()

def format_for_llama(system_prompt_text, query_text):
    """Formats a system prompt and query for the BASE Llama model pipeline."""
    messages = [
        {
            "role": "system",
            "content": system_prompt_text
        },
        {
            "role": "user",
            "content": query_text
        }
    ]
    return messages

# --- 2. Define Model/Adapter Paths ---
Llama_31_8B_IT = "meta-llama/Meta-Llama-3.1-8B-Instruct"
ADAPTER_OUTPUT_DIR_LLAMA = "/content/drive/MyDrive/Colab Notebooks/Radin/Llama_Output"

# This list will hold our dictionaries of results for Llama
llama_results_list = []
print(f"Initializing results collection for {len(test_data)} test items.")

# -----------------------------------------------------------------
# STAGE 1: INFERENCE WITH BASE MODEL (4 Prompts)
# -----------------------------------------------------------------
print("\n--- STAGE 1: Loading BASE Llama model for prompt testing ---")
base_llama_model = AutoModelForCausalLM.from_pretrained(
    Llama_31_8B_IT,
    dtype=torch.bfloat16,
    device_map="auto",
    low_cpu_mem_usage=True,
)
base_llama_tokenizer = AutoTokenizer.from_pretrained(Llama_31_8B_IT)
# Llama 3.1 requires specific pad/eos token settings for generation
if base_llama_tokenizer.pad_token is None:
    base_llama_tokenizer.pad_token = "<|eot_id|>"
    base_llama_tokenizer.eos_token = "<|eot_id|>"
    base_llama_tokenizer.pad_token_id = base_llama_tokenizer.eos_token_id

base_llama_pipe = pipeline(
    task="text-generation",
    model=base_llama_model,
    tokenizer=base_llama_tokenizer,
    dtype=torch.bfloat16
)
# Generation parameters for prompt-based inference
llama_31_8b_it_pipe_kwargs = {
    "max_new_tokens": 128,
    "return_full_text": False,
    "do_sample": True,
    "temperature": 0.8,
    "top_p": 0.9,
    "pad_token_id": base_llama_tokenizer.pad_token_id,
    "eos_token_id": base_llama_tokenizer.eos_token_id
}
print("Base Llama model loaded. Running 4 prompt configurations on test set...")

# --- Run Inference on Test Set (Base Model) ---
for index, row in tqdm(test_data.iterrows(), total=len(test_data), desc="Processing Llama (Base)"):
    content = str(row['content']).strip()
    actual_title = str(row['title']).strip()

    if not content:
        continue

    # Generate title for each of the four prompts
    formatted_simple = format_for_llama(simple_prompt, content)
    simple_gen = base_llama_pipe(formatted_simple, **llama_31_8b_it_pipe_kwargs)[0]["generated_text"]

    formatted_zero_shot = format_for_llama(zero_shot_prompt, content)
    zero_shot_gen = base_llama_pipe(formatted_zero_shot, **llama_31_8b_it_pipe_kwargs)[0]["generated_text"]

    formatted_few_shot = format_for_llama(few_shot_prompt, content)
    few_shot_gen = base_llama_pipe(formatted_few_shot, **llama_31_8b_it_pipe_kwargs)[0]["generated_text"]

    formatted_refined = format_for_llama(refined_prompt, content)
    refined_gen = base_llama_pipe(formatted_refined, **llama_31_8b_it_pipe_kwargs)[0]["generated_text"]

    # Store results, applying the cleanup function
    llama_results_list.append({
        'content': content,
        'actual_title': actual_title,
        'predicted_simple': clean_output(simple_gen),
        'predicted_zero_shot': clean_output(zero_shot_gen),
        'predicted_few_shot': clean_output(few_shot_gen),
        'predicted_refined': clean_output(refined_gen),
    })

# --- Cleanup Stage 1 ---
print("Base model inference complete. Clearing memory...")
del base_llama_model, base_llama_pipe, base_llama_tokenizer
gc.collect()
torch.cuda.empty_cache()
print("Memory cleared.")

# -----------------------------------------------------------------
# STAGE 2: INFERENCE WITH FINE-TUNED MODEL (1 Prompt)
# -----------------------------------------------------------------
print("\n--- STAGE 2: Loading FINE-TUNED Llama model and LoRA adapter ---")

# Load base model again to attach adapter
ft_llama_base_model = AutoModelForCausalLM.from_pretrained(
    Llama_31_8B_IT,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    low_cpu_mem_usage=True,
)
# Attach the trained LoRA adapter weights
fine_tuned_llama_model = PeftModel.from_pretrained(
    ft_llama_base_model,
    os.path.join(ADAPTER_OUTPUT_DIR_LLAMA, "final_checkpoint")
)
ft_llama_tokenizer = AutoTokenizer.from_pretrained(Llama_31_8B_IT)
# Set special tokens again for the fine-tuned tokenizer
if ft_llama_tokenizer.pad_token is None:
    ft_llama_tokenizer.pad_token = "<|eot_id|>"
if ft_llama_tokenizer.eos_token is None:
    ft_llama_tokenizer.eos_token = "<|eot_id|>"
ft_llama_tokenizer.pad_token_id = ft_llama_tokenizer.eos_token_id

ft_llama_pipe = pipeline(
    task="text-generation",
    model=fine_tuned_llama_model,
    tokenizer=ft_llama_tokenizer,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)
# Generation parameters for fine-tuned inference
ft_llama_pipe_kwargs = {
    "max_new_tokens": 50,
    "do_sample": True,
    "temperature": 0.7,
    "top_p": 0.9,
    "num_return_sequences": 1,
    "pad_token_id": ft_llama_tokenizer.pad_token_id,
    "eos_token_id": ft_llama_tokenizer.eos_token_id,
    "return_full_text": False,
}
print("Fine-tuned Llama model loaded. Running fine-tuning prompt...")

# --- Run Inference on Test Set (Fine-Tuned Model) ---
# Iterate through the pre-populated results list to add the final column
for i in tqdm(range(len(llama_results_list)), desc="Processing Llama (FT)"):
    content = llama_results_list[i]['content'] # Get content from our stored list

    # Format input using the tokenizer's chat template for the fine-tuned model
    messages_ft = [
        {"role": "system", "content": fine_tuning_system_prompt},
        {"role": "user", "content": content},
    ]
    formatted_input_ft = ft_llama_tokenizer.apply_chat_template(
        messages_ft, tokenize=False, add_generation_prompt=True
    )
    # Generate the prediction
    ft_gen = ft_llama_pipe(formatted_input_ft, **ft_llama_pipe_kwargs)[0]["generated_text"]

    # Add the fine-tuned prediction to the existing dictionary
    llama_results_list[i]['predicted_fine_tuned'] = clean_output(ft_gen)

print("Fine-tuned inference complete.")

# --- Cleanup Stage 2 ---
print("Cleaning up Llama models from memory...")
del ft_llama_base_model, fine_tuned_llama_model, ft_llama_pipe, ft_llama_tokenizer
gc.collect()
torch.cuda.empty_cache()
print("Memory cleared.")

# -----------------------------------------------------------------
# STAGE 3: SAVE RESULTS
# -----------------------------------------------------------------
print("\n--- STAGE 3: Saving all results to CSV ---")
# Convert the list of results into a DataFrame and save
llama_df = pd.DataFrame(llama_results_list)
llama_df.to_csv("llama_results.csv", index=False, encoding='utf-8-sig')

print("\n" + "="*50)
print(f"✅ Llama results saved to llama_results.csv")
print("First 5 rows of Llama results:")
print(llama_df.head())
print("="*50 + "\n")
print("\nAll Llama inference tasks are finished.")

## Step 4.3: Quantitative Evaluation Metrics

This section establishes the framework for rigorously evaluating the quality of the generated titles using four industry-standard metrics: **ROUGE-L** (Overlap-based, Recall/Precision/F1), **BLEU** (Precision-based, $n$-gram match), **METEOR** (Semantic and $n$-gram alignment), and **BERTScore** (Embeddings-based semantic similarity).

To handle the Persian language correctly:
* **Preprocessing:** The `hazm.Normalizer` and `word_tokenize` are utilized to standardize and segment both the predicted and actual titles.
* **SacreBLEU:** The robust `sacrebleu` library is used with the **`flores200`** tokenizer, which is highly recommended for cross-lingual tasks involving resource-poor languages, ensuring more accurate tokenization than simple splitting.
* **BERTScore:** The `lang="fa"` parameter is specified to use a Persian-specific BERT model for embedding comparison.

A master function, `evaluate_model_predictions`, processes all results, calculates all four scores per prediction, saves detailed row-level results, and returns a summarized average performance table.

In [3]:
# --- 1. Initialize Persian Tools and ROUGE Scorer ---
from hazm import Normalizer, word_tokenize
from rouge_score import rouge_scorer
from nltk.translate.meteor_score import meteor_score
from bert_score import score as bert_score_calc
import sacrebleu

# Initialize the hazm normalizer for standardizing Persian text
normalizer = Normalizer()

# Custom Tokenizer Wrapper required by the `rouge_scorer` object
class HazmTokenizer:
    def tokenize(self, text):
        # Use hazm's word_tokenize for accurate Persian segmentation
        return word_tokenize(text)

hazm_tokenizer = HazmTokenizer()
# Initialize ROUGE-L scorer, disabling internal stemming and passing the Hazm tokenizer
rouge_scorer_obj = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=False, tokenizer=hazm_tokenizer)

# --- 2. Separated Preprocessing Functions ---
def normalize_text(text: str) -> str:
    """Applies hazm normalization and stripping to input text."""
    if not isinstance(text, str): return ""
    return normalizer.normalize(text.strip())

def tokenize_text(text: str) -> list:
    """Tokenizes a string using hazm's word_tokenize (used for ROUGE and METEOR)."""
    if not isinstance(text, str): return []
    return word_tokenize(text)

# --- 3. Master Evaluation Function ---
def evaluate_model_predictions(df: pd.DataFrame, model_name: str = "Model") -> pd.DataFrame:
    """
    Calculates ROUGE-L, BLEU (sacrebleu), METEOR, and BERTScore for all predicted columns
    in the DataFrame against the 'actual_title' column.
    """
    pred_cols = [col for col in df.columns if col.startswith('predicted_')]
    actuals_raw = df['actual_title'].astype(str).tolist()

    print(f"📉 Preprocessing {len(actuals_raw)} reference titles...")
    # Preprocess references once
    refs_normalized = [normalize_text(ref) for ref in actuals_raw]
    refs_tokenized = [tokenize_text(norm_ref) for norm_ref in refs_normalized]

    summary_results = {}

    for col in pred_cols:
        print(f"\n🔍 Evaluating column: {col}...")
        preds_raw = df[col].astype(str).tolist()

        # Preprocess predictions
        preds_normalized = [normalize_text(pred) for pred in preds_raw]
        preds_tokenized = [tokenize_text(norm_pred) for norm_pred in preds_normalized]

        print(f"   ⏳ Calculating BERTScore for {col}...")
        # BERTScore calculation (takes the raw strings, lang="fa" uses a Persian checkpoint)
        P, R, F1 = bert_score_calc(preds_raw, actuals_raw, lang="fa", verbose=False, device=None, batch_size=32)
        bert_scores_list = F1.numpy().tolist()

        row_level_results = []
        print(f"   ... Calculating lexical and semantic scores for {len(actuals_raw)} rows.")

        for i in tqdm(range(len(actuals_raw)), desc="Metric Calculation"):

            # --- SacreBLEU (using Flores200 tokenizer) ---
            # SacreBLEU works best on normalized strings with an explicit tokenizer.
            bleu_score = sacrebleu.sentence_bleu(
                preds_normalized[i],       # Normalized prediction string
                [refs_normalized[i]],      # List containing the single normalized reference string
                tokenize='flores200'       # Recommended tokenizer for robust language-agnostic scoring
            ).score
            # SacreBLEU returns 0-100, normalize to 0-1
            bleu_score = bleu_score / 100.0

            # --- ROUGE-L ---
            # Uses hazm tokenizer initialized earlier
            rouge_l_score = rouge_scorer_obj.score(refs_normalized[i], preds_normalized[i])['rougeL'].fmeasure

            # --- METEOR ---
            # Requires tokenized inputs
            meteor_score_val = meteor_score([refs_tokenized[i]], preds_tokenized[i])

            # --- BERTScore (pre-calculated) ---
            bert_score = bert_scores_list[i]

            row_level_results.append({
                "actual_title": actuals_raw[i],
                "predicted_title": preds_raw[i],
                "BLEU": bleu_score,
                "ROUGE-L": rouge_l_score,
                "METEOR": meteor_score_val,
                "BERTScore": bert_score
            })

        col_df = pd.DataFrame(row_level_results)
        output_filename = f"{model_name}_{col}_scores.csv"
        # Save the detailed row-level scores
        col_df.to_csv(output_filename, index=False, encoding='utf-8-sig')
        print(f"   ✅ Saved detailed scores to {output_filename}")

        # Calculate mean scores for summary table
        summary_results[col] = {
            "BLEU": col_df["BLEU"].mean(),
            "ROUGE-L": col_df["ROUGE-L"].mean(),
            "METEOR": col_df["METEOR"].mean(),
            "BERTScore": col_df["BERTScore"].mean()
        }

    metrics_df = pd.DataFrame(summary_results).T
    metrics_df.index.name = f"{model_name} Prompt Types"

    return metrics_df

### Step 4.4: Evaluate Gemma Results

The generated titles for the Gemma model across all five configurations are loaded from the saved CSV file. The `evaluate_model_predictions` function is called to calculate the average performance for the four metrics (BLEU, ROUGE-L, METEOR, BERTScore) for each prompt and the fine-tuned adapter. The final summary table is displayed and saved.

In [ ]:
# Load results
try:
    gemma_results_df = pd.read_csv("gemma_results.csv")
    print(f"Loaded Gemma results with {len(gemma_results_df)} rows.")

    # Run Evaluation
    gemma_metrics = evaluate_model_predictions(gemma_results_df, model_name="Gemma-3-4B")

    print("\n🏆 --- FINAL EVALUATION RESULTS: GEMMA-3-4B (Averages) --- 🏆")
    # Display the final results table with formatting
    display(gemma_metrics.style.background_gradient(cmap='viridis', axis=0).format("{:.4f}"))

    # Save metrics to CSV for external analysis
    gemma_metrics.to_csv("gemma_evaluation_metrics.csv")

except FileNotFoundError:
    print("❌ 'gemma_results.csv' not found. Please run the inference cells first.")

### Step 4.5: Evaluate Llama Results

The generated titles for the Llama model are loaded. The same evaluation function is applied to calculate the average metric performance across all configurations. This allows for a direct, quantitative comparison between the Llama and Gemma models, benchmarking the effectiveness of the initial instruction-tuning, the prompt engineering, and the LoRA fine-tuning for this specific task. The final summary table is displayed and saved.

In [ ]:
# Load results
try:
    llama_results_df = pd.read_csv("llama_results.csv")
    print(f"Loaded Llama results with {len(llama_results_df)} rows.")

    # Run Evaluation
    llama_metrics = evaluate_model_predictions(llama_results_df, model_name="Llama-3.1-8B")

    print("\n🏆 --- FINAL EVALUATION RESULTS: LLAMA-3.1-8B (Averages) --- 🏆")
    # Display the final results table with formatting
    display(llama_metrics.style.background_gradient(cmap='viridis', axis=0).format("{:.4f}"))

    # Save metrics to CSV for external analysis
    llama_metrics.to_csv("llama_evaluation_metrics.csv")

except FileNotFoundError:
    print("❌ 'llama_results.csv' not found. Please run the inference cells first.")